### Loading Required Modules

In [1]:
"""
Loading Required Modules
"""
from pdfminer3.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer3.converter import TextConverter
from pdfminer3.layout import LAParams
from pdfminer3.pdfpage import PDFPage
from io import BytesIO
from io import StringIO
import os
import pandas as pd
import numpy as np

## Keyword Search Utility 
from flashtext import KeywordProcessor
import re
import string

from cleanco import cleanco
import re 
import string

import pdf_to_html as pth
import multiprocessing as mp

In [2]:
"""
Define The Paths
"""
RAW_DATA_PATH = "../2. Data/raw"
HTML_PATH = "../2. Data/html"
TXT_PATH = "../2. Data/txt"
PROT_DF_PATH =  "../2. Data/prot_df"

### Helper Functions

In [3]:
def convertPDFToText_String(path):
    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr,codec=codec,laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos=set()
        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)
        fp.close()
        device.close()
        string = retstr.getvalue()
        retstr.close()
        return string
    except Exception as e:
        print(traceback.format_exc())
        print(e)
        return "Fail"

In [4]:
def clean_text_func(text):
        text = str(text)
        text = text.lower()
        text = re.sub(r'[^\x00-\x7F]+',' ', text)
        text = text.replace("\n", " ")
        translator = re.compile('[%s]' % re.escape(string.punctuation))
        text = translator.sub(' ', text)
        text = re.sub(' +', ' ', text)
        text = " ".join(text.split())
        text = text.strip()
        return text

class basic_keyword_extract:
    """Basic keyword extractor class using the keyword processor by Flashtext"""
    def __init__(self, keyword_list, case_sensitive=False):
        self.keyword_list = keyword_list
        self.case_sensitive = case_sensitive
        self.keyword_processor = self._initialize_basic_keyword_processor()
   
    def _initialize_basic_keyword_processor(self):
        kp = KeywordProcessor(case_sensitive=self.case_sensitive)
        kp.add_keywords_from_list(self.keyword_list)
        return kp
    def extract_from_text(self, raw_text):
        if self.case_sensitive:
            text = raw_text
        else:
            text = raw_text.lower().strip()
            clean_text = clean_text_func(text)
            
        clean_extracted_keywords_ls = self.keyword_processor.extract_keywords(clean_text, span_info=True)
#         output_ls = list(set([i[0] for i in clean_extracted_keywords_ls]))
        output_ls = [{'keyword':i[0], 'start_idx':i[1], 'end_idx':i[2]} for i in clean_extracted_keywords_ls]
#         output_ls = list({i['keyword']:i for i in reversed(output_ls)}.values()) ## Remove dups
        if len(output_ls) == 0:
            return []
        else:
            return output_ls

In [5]:
def clean_company_name(text):
    
    text = cleanco(text).clean_name()
    text = str(text)
    text = text.lower()
    text = text.lower().replace(' mfg. co. ltd.', ' ').replace('group', '').replace(
                ' & co. ltd.', ' ').replace(' co. ltd.', ' ').replace(' ltd.', ' ').replace('ltd', '').replace('corporation', '').replace(
                'limited', ' ').replace(' pvt.', ' ').replace('pvt', '').replace('company', ' ').replace('india', '').replace('private', '').strip()
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.replace("\n", " ")
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    text = translator.sub(' ', text)
    text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    text = text.strip()
    return text

In [6]:
def clean_special_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.replace("\n", " ")
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    text = translator.sub(' ', text)
    text = re.sub(r'(?<=\b[a-z]) (?=[a-z]\b)', '', text)
    text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    text = text.strip()
    return text

In [7]:
def clean_ner_text(text):
    text = str(text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.replace("\n", " ")
    text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    return text

### Experience

#### Company Extraction

In [8]:
## Company Names
file = "../2. Data/indian_companies_list_v5.txt"
file = open(file, "r", encoding='utf-8')

## Create Company List
COMPANIES_LIST = [line.strip().lower() for line in file]

In [9]:
## Initialize Keyword Processor:
COMPANY_KEYWORDS_SEARCH = basic_keyword_extract(COMPANIES_LIST, case_sensitive=False)

In [10]:
## Function to extract company names
def get_company_names(text):
    text = clean_special_text(text)
    company_ls = COMPANY_KEYWORDS_SEARCH.extract_from_text(text)
    return company_ls

In [ ]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    company = get_company_names(text)
    print("Companies: ", company, end="\n---------------------------------------------- \n")

In [12]:
# company_ls = list(set(company_ls) - set(remove_ls))

# with open('../2. Data/indian_companies_list_v6.txt', 'w', encoding='utf-8') as f:
#     for item in company_ls:
#         f.write("%s\n" % item)

#### Designation Extraction

In [11]:
## Company Names
file = "../2. Data/job_titles_list_v2.txt"
file = open(file, "r", encoding='utf-8')

TITLE_LIST = [line.strip().lower() for line in file]

In [12]:
## Initialize Keyword Processor:
TITLE_KEYWORDS_SEARCH = basic_keyword_extract(TITLE_LIST, case_sensitive=False)

In [13]:
def get_job_titles(text):
    text = clean_special_text(text)
    title_ls = TITLE_KEYWORDS_SEARCH.extract_from_text(text)
    return title_ls

In [16]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    designation = get_job_titles(text)
    print("Designation: ", designation, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Designation:  [{'keyword': 'vice president', 'start_idx': 6515, 'end_idx': 6529}, {'keyword': 'assistant manager', 'start_idx': 5624, 'end_idx': 5641}, {'keyword': 'senior consultant', 'start_idx': 423, 'end_idx': 440}, {'keyword': 'project manager', 'start_idx': 2711, 'end_idx': 2726}, {'keyword': 'finance professional', 'start_idx': 59, 'end_idx': 79}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
Designation:  [{'keyword': 'consultant', 'start_idx': 4325, 'end_idx': 4335}, {'keyword': 'credit manager', 'start_idx': 858, 'end_idx': 872}, {'keyword': 'chartered accountant', 'start_idx': 122, 'end_idx': 142}]
---------------------------------------------- 
FileName :  Abhilash Pillai- CV.pdf
Designation:  [{'keyword': 'director', 'start_idx': 16, 'end_idx': 24}, {'keyword': 'senior software developer', 'start_idx': 5256, 'end_idx': 5281}, {'keyword': 'sales marketing', 'start_idx': 4829, 'end_idx': 4844}, {'keyword': '

Designation:  [{'keyword': 'operations analyst', 'start_idx': 7618, 'end_idx': 7636}, {'keyword': 'management trainee', 'start_idx': 7568, 'end_idx': 7586}, {'keyword': 'senior analyst', 'start_idx': 5583, 'end_idx': 5597}, {'keyword': 'securities analyst', 'start_idx': 6050, 'end_idx': 6068}, {'keyword': 'associate', 'start_idx': 4561, 'end_idx': 4570}, {'keyword': 'operator', 'start_idx': 4153, 'end_idx': 4161}, {'keyword': 'cio', 'start_idx': 2352, 'end_idx': 2355}, {'keyword': 'portfolio manager', 'start_idx': 2183, 'end_idx': 2200}, {'keyword': 'founder', 'start_idx': 1806, 'end_idx': 1813}, {'keyword': 'chairman', 'start_idx': 1793, 'end_idx': 1801}, {'keyword': 'senior vice president', 'start_idx': 119, 'end_idx': 140}]
---------------------------------------------- 
FileName :  Amit Singh- CV.pdf
Designation:  [{'keyword': 'legal counsel', 'start_idx': 39, 'end_idx': 52}, {'keyword': 'legal officer', 'start_idx': 3998, 'end_idx': 4011}, {'keyword': 'manager legal', 'start_idx':

Designation:  [{'keyword': 'blog writer', 'start_idx': 1817, 'end_idx': 1828}, {'keyword': 'research intern', 'start_idx': 1256, 'end_idx': 1271}, {'keyword': 'science intern', 'start_idx': 1128, 'end_idx': 1142}, {'keyword': 'data scientist', 'start_idx': 256, 'end_idx': 270}]
---------------------------------------------- 
FileName :  Anshuman Kumar Rath- CV.pdf
Designation:  [{'keyword': 'hr business partner', 'start_idx': 5203, 'end_idx': 5222}, {'keyword': 'process owner', 'start_idx': 3793, 'end_idx': 3806}, {'keyword': 'project manager', 'start_idx': 3777, 'end_idx': 3792}, {'keyword': 'graduate engineer', 'start_idx': 1907, 'end_idx': 1924}, {'keyword': 'assistant manager', 'start_idx': 1886, 'end_idx': 1903}, {'keyword': 'hr manager', 'start_idx': 1739, 'end_idx': 1749}, {'keyword': 'senior consultant', 'start_idx': 1721, 'end_idx': 1738}, {'keyword': 'vice president', 'start_idx': 1395, 'end_idx': 1409}, {'keyword': 'human resources professional', 'start_idx': 275, 'end_idx':

Designation:  [{'keyword': 'sr manager', 'start_idx': 7923, 'end_idx': 7933}, {'keyword': 'senior manager', 'start_idx': 7870, 'end_idx': 7884}, {'keyword': 'management trainee', 'start_idx': 7467, 'end_idx': 7485}, {'keyword': 'hr business partner', 'start_idx': 4483, 'end_idx': 4502}, {'keyword': 'sales operations', 'start_idx': 6613, 'end_idx': 6629}, {'keyword': 'regional hr manager', 'start_idx': 1487, 'end_idx': 1506}, {'keyword': 'human resources manager', 'start_idx': 5962, 'end_idx': 5985}, {'keyword': 'practice managers', 'start_idx': 5513, 'end_idx': 5530}, {'keyword': 'process owner', 'start_idx': 4455, 'end_idx': 4468}, {'keyword': 'director', 'start_idx': 2201, 'end_idx': 2209}, {'keyword': 'research manager', 'start_idx': 4147, 'end_idx': 4163}, {'keyword': 'project leader', 'start_idx': 3204, 'end_idx': 3218}, {'keyword': 'executive director', 'start_idx': 3130, 'end_idx': 3148}, {'keyword': 'associate director', 'start_idx': 1259, 'end_idx': 1277}, {'keyword': 'ceo', '

Designation:  [{'keyword': 'advisor consultant', 'start_idx': 4019, 'end_idx': 4037}, {'keyword': 'co founder', 'start_idx': 564, 'end_idx': 574}, {'keyword': 'vp', 'start_idx': 3412, 'end_idx': 3414}, {'keyword': 'managing director', 'start_idx': 2705, 'end_idx': 2722}, {'keyword': 'director', 'start_idx': 36, 'end_idx': 44}, {'keyword': 'vice president', 'start_idx': 1118, 'end_idx': 1132}, {'keyword': 'senior product manager', 'start_idx': 935, 'end_idx': 957}, {'keyword': 'owner', 'start_idx': 871, 'end_idx': 876}, {'keyword': 'avp', 'start_idx': 775, 'end_idx': 778}, {'keyword': 'product development', 'start_idx': 755, 'end_idx': 774}, {'keyword': 'consultant', 'start_idx': 506, 'end_idx': 516}]
---------------------------------------------- 
FileName :  BARNALI ROYCHOWDHURY- CV.pdf
Designation:  [{'keyword': 'management trainee', 'start_idx': 6376, 'end_idx': 6394}, {'keyword': 'ceo', 'start_idx': 743, 'end_idx': 746}, {'keyword': 'recruitment specialist', 'start_idx': 2511, 'end

...

Designation:  [{'keyword': 'owner', 'start_idx': 1422, 'end_idx': 1427}, {'keyword': 'director', 'start_idx': 1240, 'end_idx': 1248}, {'keyword': 'management consultant', 'start_idx': 197, 'end_idx': 218}]
---------------------------------------------- 
FileName :  Shriram Krishnan- CV.pdf
Designation:  [{'keyword': 'finance professional', 'start_idx': 19, 'end_idx': 39}, {'keyword': 'counsel', 'start_idx': 425, 'end_idx': 432}, {'keyword': 'vp', 'start_idx': 253, 'end_idx': 255}, {'keyword': 'cfo', 'start_idx': 123, 'end_idx': 126}, {'keyword': 'consultant', 'start_idx': 59, 'end_idx': 69}]
---------------------------------------------- 
FileName :  SHUBHANGA PRASAD- CV.pdf
Designation:  [{'keyword': 'team lead', 'start_idx': 3853, 'end_idx': 3862}, {'keyword': 'faculty member', 'start_idx': 3020, 'end_idx': 3034}, {'keyword': 'product development', 'start_idx': 2652, 'end_idx': 2671}, {'keyword': 'partner', 'start_idx': 1458, 'end_idx': 1465}, {'keyword': 'digital tech', 'start_idx':

Designation:  [{'keyword': 'certified public accountant', 'start_idx': 2312, 'end_idx': 2339}, {'keyword': 'company secretary', 'start_idx': 1552, 'end_idx': 1569}, {'keyword': 'chartered accountant', 'start_idx': 2075, 'end_idx': 2095}, {'keyword': 'management trainee', 'start_idx': 1956, 'end_idx': 1974}, {'keyword': 'finance executive', 'start_idx': 1884, 'end_idx': 1901}, {'keyword': 'avp', 'start_idx': 1697, 'end_idx': 1700}, {'keyword': 'vp', 'start_idx': 1694, 'end_idx': 1696}, {'keyword': 'financial controller', 'start_idx': 1531, 'end_idx': 1551}, {'keyword': 'cfo', 'start_idx': 1461, 'end_idx': 1464}, {'keyword': 'finance professional', 'start_idx': 139, 'end_idx': 159}]
---------------------------------------------- 
FileName :  Sujan Hajra- CV.pdf
Designation:  [{'keyword': 'executive director', 'start_idx': 12, 'end_idx': 30}, {'keyword': 'director', 'start_idx': 1923, 'end_idx': 1931}, {'keyword': 'vp', 'start_idx': 1867, 'end_idx': 1869}, {'keyword': 'senior vice preside

Designation:  [{'keyword': 'company secretary', 'start_idx': 6503, 'end_idx': 6520}, {'keyword': 'cfo', 'start_idx': 5326, 'end_idx': 5329}, {'keyword': 'manager financial planning', 'start_idx': 873, 'end_idx': 899}, {'keyword': 'business advisor', 'start_idx': 1448, 'end_idx': 1464}, {'keyword': 'financial controller', 'start_idx': 826, 'end_idx': 846}]
---------------------------------------------- 
FileName :  Tejash Mahendra Tanna- CV.pdf
Designation:  [{'keyword': 'manager internal', 'start_idx': 4431, 'end_idx': 4447}, {'keyword': 'cfo', 'start_idx': 369, 'end_idx': 372}, {'keyword': 'chief financial officer', 'start_idx': 345, 'end_idx': 368}, {'keyword': 'chartered accountant', 'start_idx': 22, 'end_idx': 42}]
---------------------------------------------- 
FileName :  Uddeeptabh Mishra- CV (Kelly Services).pdf
Designation:  [{'keyword': 'learning designer', 'start_idx': 5006, 'end_idx': 5023}, {'keyword': 'process owner', 'start_idx': 3262, 'end_idx': 3275}, {'keyword': 'mana

Designation:  [{'keyword': 'regional director', 'start_idx': 22654, 'end_idx': 22671}, {'keyword': 'chief financial officer', 'start_idx': 20774, 'end_idx': 20797}, {'keyword': 'financial controller', 'start_idx': 3563, 'end_idx': 3583}, {'keyword': 'deputy manager', 'start_idx': 3303, 'end_idx': 3317}, {'keyword': 'pricing coordinator', 'start_idx': 19909, 'end_idx': 19928}, {'keyword': 'site controller', 'start_idx': 3010, 'end_idx': 3025}, {'keyword': 'support director', 'start_idx': 15591, 'end_idx': 15607}, {'keyword': 'partner', 'start_idx': 15228, 'end_idx': 15235}, {'keyword': 'business controller', 'start_idx': 2956, 'end_idx': 2975}, {'keyword': 'director', 'start_idx': 539, 'end_idx': 547}, {'keyword': 'sales operations', 'start_idx': 11388, 'end_idx': 11404}, {'keyword': 'cfo', 'start_idx': 565, 'end_idx': 568}, {'keyword': 'site director', 'start_idx': 7709, 'end_idx': 7722}, {'keyword': 'co pilot', 'start_idx': 3851, 'end_idx': 3859}, {'keyword': 'chartered accountant', '

Designation:  [{'keyword': 'tax consultant', 'start_idx': 1957, 'end_idx': 1971}]
---------------------------------------------- 
FileName :  Yatin Joshi- CV.pdf
Designation:  [{'keyword': 'sales manager', 'start_idx': 4946, 'end_idx': 4959}, {'keyword': 'business manager', 'start_idx': 6981, 'end_idx': 6997}, {'keyword': 'technical manager', 'start_idx': 5996, 'end_idx': 6013}, {'keyword': 'sales and marketing manager', 'start_idx': 5705, 'end_idx': 5732}, {'keyword': 'area sales manager', 'start_idx': 5540, 'end_idx': 5558}, {'keyword': 'product development', 'start_idx': 3398, 'end_idx': 3417}, {'keyword': 'national sales manager', 'start_idx': 4172, 'end_idx': 4194}, {'keyword': 'business development manager', 'start_idx': 3723, 'end_idx': 3751}, {'keyword': 'sales marketing', 'start_idx': 96, 'end_idx': 111}, {'keyword': 'area manager', 'start_idx': 2906, 'end_idx': 2918}, {'keyword': 'sales marketing manager', 'start_idx': 2371, 'end_idx': 2394}]
---------------------------------

In [17]:
# title_ls = list(set(title_ls) - set(remove_ls))

# with open('../2. Data/job_titles_list_v2.txt', 'w', encoding='utf-8') as f:
#     for item in title_ls:
#         f.write("%s\n" % item)

#### Time Period Extraction

In [225]:
def get_period(text):
    date_ls = []
    clean_txt = clean_special_text(text)
    for txt in text.split("\n"):
        try:
            date_dict = {'period': '', 'start_idx': '', 'end_idx': ''}
            search_txt = clean_special_text(txt)
            search_out = DATE_KEYWORDS_SEARCH.extract_from_text(search_txt)
            if len(search_out) > 0:
                match = (re.search(search_txt, clean_txt))
                date_dict['period'] = search_txt
                date_dict['start_idx'] = match.span()[0]
                date_dict['end_idx'] = match.span()[1]
                date_ls.append(date_dict)
        except Exception as e:
            continue
    
    return date_ls

In [208]:
start_year = 1947
end_year = int(datetime.datetime.now().year)
year_ls = [str(i) for i in np.arange(start_year, end_year)]

DATE_KEYWORDS_LS = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec',
                    'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november',
                    'december',
                    'date', 'present'] + year_ls

In [209]:
## Initialize Keyword Processor:
DATE_KEYWORDS_SEARCH = basic_keyword_extract(DATE_KEYWORDS_LS, case_sensitive=False)

In [83]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    tenure_ls = get_period(text)
    print("Time Period: ", tenure_ls, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Time Period:  [{'period': 'Nov 2012', 'start_char': 4198, 'end_char': 4206}, {'period': '2013 Monthly', 'start_char': 4211, 'end_char': 4223}, {'period': 'May 2009', 'start_char': 7605, 'end_char': 7613}, {'period': 'May 2004', 'start_char': 7620, 'end_char': 7628}, {'period': 'May 2004', 'start_char': 8726, 'end_char': 8734}, {'period': '2005', 'start_char': 9081, 'end_char': 9085}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
Time Period:  [{'period': 'April 2017- February 2018', 'start_char': 726, 'end_char': 751}, {'period': 'July 2015 April 2017', 'start_char': 1033, 'end_char': 1053}, {'period': 'April 2011 - October 2014', 'start_char': 1975, 'end_char': 2000}, {'period': 'October 2003-March', 'start_char': 3244, 'end_char': 3262}, {'period': 'the years of 2008', 'start_char': 4114, 'end_char': 4131}, {'period': '2009', 'start_char': 4136, 'end_char': 4140}, {'period': 'September 2002 - October 2003', 'start_ch

Time Period:  [{'period': 'May, 2004', 'start_char': 550, 'end_char': 559}, {'period': '2012', 'start_char': 629, 'end_char': 633}, {'period': '2015', 'start_char': 842, 'end_char': 846}, {'period': 'Oct 2015', 'start_char': 941, 'end_char': 949}, {'period': 'May 2011', 'start_char': 1309, 'end_char': 1317}, {'period': 'Oct 2015', 'start_char': 1757, 'end_char': 1765}]
---------------------------------------------- 
FileName :  Aditya Narayan Shetty- CV.pdf
Time Period:  [{'period': 'August 2011', 'start_char': 757, 'end_char': 768}, {'period': 'August 2011', 'start_char': 1027, 'end_char': 1038}, {'period': 'November 2009 to July 2011', 'start_char': 2831, 'end_char': 2857}, {'period': 'May 2012', 'start_char': 4413, 'end_char': 4421}, {'period': '[2002 to 2005', 'start_char': 4778, 'end_char': 4791}, {'period': '2000 to 2002', 'start_char': 4900, 'end_char': 4912}, {'period': 'December 2003', 'start_char': 5117, 'end_char': 5130}, {'period': 'February 27, 1985', 'start_char': 5366, '

Time Period:  [{'period': '2012-13', 'start_char': 5715, 'end_char': 5722}, {'period': '2012-13', 'start_char': 6044, 'end_char': 6051}, {'period': '2012-13', 'start_char': 6231, 'end_char': 6238}, {'period': 'between 2012-13 & 2013-14', 'start_char': 6581, 'end_char': 6606}, {'period': '2012-13', 'start_char': 6666, 'end_char': 6673}, {'period': '2012-13', 'start_char': 6713, 'end_char': 6720}, {'period': '1998', 'start_char': 7807, 'end_char': 7811}, {'period': '1994', 'start_char': 7890, 'end_char': 7894}]
---------------------------------------------- 
FileName :  Anchal Sultania- CV.pdf
Time Period:  [{'period': '2001', 'start_char': 10987, 'end_char': 10991}, {'period': 'March 2010', 'start_char': 11570, 'end_char': 11580}, {'period': 'Sept Dec 2009 July 2005', 'start_char': 11581, 'end_char': 11604}]
---------------------------------------------- 
FileName :  Andrew Stansbie- CV (Hospitale Recruitment).pdf
Time Period:  [{'period': 'August 2007', 'start_char': 328, 'end_char': 3

Time Period:  [{'period': 'August 2013', 'start_char': 514, 'end_char': 525}, {'period': 'December 2014', 'start_char': 828, 'end_char': 841}, {'period': 'May 2016', 'start_char': 1024, 'end_char': 1032}, {'period': 'May / June 2017', 'start_char': 1786, 'end_char': 1801}, {'period': '2008', 'start_char': 2316, 'end_char': 2320}, {'period': '2009', 'start_char': 2325, 'end_char': 2329}]
---------------------------------------------- 
FileName :  ANSHUL KUMAR GUPTA- CV (Rinalytics Advisors).pdf
Time Period:  [{'period': '2012-2016', 'start_char': 80, 'end_char': 89}, {'period': '2011 2009', 'start_char': 121, 'end_char': 130}, {'period': '2013', 'start_char': 1965, 'end_char': 1969}]
---------------------------------------------- 
FileName :  Anshuman Kumar Rath- CV.pdf
Time Period:  [{'period': 'May 2014', 'start_char': 1603, 'end_char': 1611}, {'period': 'April 2015', 'start_char': 1624, 'end_char': 1634}, {'period': 'April 2007', 'start_char': 1691, 'end_char': 1701}, {'period': 'May

Time Period:  [{'period': '2015', 'start_char': 81, 'end_char': 85}, {'period': '2012', 'start_char': 2625, 'end_char': 2629}, {'period': 'July 2009 to March 2012', 'start_char': 5221, 'end_char': 5244}, {'period': '2004', 'start_char': 7468, 'end_char': 7472}, {'period': 'Jan 2008', 'start_char': 7476, 'end_char': 7484}, {'period': 'May 2002', 'start_char': 8477, 'end_char': 8485}, {'period': 'June 1998', 'start_char': 8768, 'end_char': 8777}, {'period': 'May 2002', 'start_char': 8781, 'end_char': 8789}, {'period': '1995', 'start_char': 8837, 'end_char': 8841}, {'period': 'May 1998', 'start_char': 9016, 'end_char': 9024}, {'period': '1992', 'start_char': 9139, 'end_char': 9143}, {'period': '1990', 'start_char': 9203, 'end_char': 9207}, {'period': '25TH August 1974', 'start_char': 9234, 'end_char': 9250}]
---------------------------------------------- 
FileName :  ARNOV KUMAR- CV.pdf
Time Period:  [{'period': 'April 2011', 'start_char': 279, 'end_char': 289}, {'period': 'December 2015'

...

Time Period:  [{'period': 'during1989-1990', 'start_char': 11089, 'end_char': 11104}]
---------------------------------------------- 
FileName :  Srikanth Nadhamuni- CV.pdf
Time Period:  [{'period': '2003', 'start_char': 814, 'end_char': 818}, {'period': 'September 2016 - Present', 'start_char': 1357, 'end_char': 1381}, {'period': 'October 2013', 'start_char': 1452, 'end_char': 1464}, {'period': 'October 2012', 'start_char': 1526, 'end_char': 1538}, {'period': 'July 2009 - September 2012', 'start_char': 1619, 'end_char': 1645}, {'period': 'March 2003 - June 2009', 'start_char': 1939, 'end_char': 1961}, {'period': 'January 2000', 'start_char': 2014, 'end_char': 2026}, {'period': 'February 1996', 'start_char': 2097, 'end_char': 2110}, {'period': 'July 1994 - February 1996', 'start_char': 2186, 'end_char': 2211}, {'period': 'February 1993 - March 1994', 'start_char': 2266, 'end_char': 2292}, {'period': 'January 1989 - February 1993', 'start_char': 2356, 'end_char': 2384}, {'period': '1986

Time Period:  [{'period': 'Dec 2013', 'start_char': 1285, 'end_char': 1293}, {'period': 'Jan 2011 Jan 2013', 'start_char': 2547, 'end_char': 2564}, {'period': '2011', 'start_char': 3105, 'end_char': 3109}, {'period': '2011', 'start_char': 3147, 'end_char': 3151}, {'period': '2012', 'start_char': 3230, 'end_char': 3234}, {'period': '2012', 'start_char': 3319, 'end_char': 3323}, {'period': 'Dec 2010', 'start_char': 3333, 'end_char': 3341}, {'period': 'May 2000', 'start_char': 4854, 'end_char': 4862}, {'period': 'Oct 2004', 'start_char': 4863, 'end_char': 4871}, {'period': '2000', 'start_char': 7278, 'end_char': 7282}, {'period': '1999', 'start_char': 7546, 'end_char': 7550}, {'period': '1997', 'start_char': 9025, 'end_char': 9029}, {'period': '1996', 'start_char': 9198, 'end_char': 9202}, {'period': '2010', 'start_char': 9354, 'end_char': 9358}, {'period': '2009', 'start_char': 9422, 'end_char': 9426}, {'period': '2007', 'start_char': 9534, 'end_char': 9538}, {'period': '2015', 'start_ch

Time Period:  [{'period': '2016 June 2017', 'start_char': 553, 'end_char': 567}, {'period': 'July 2014', 'start_char': 2794, 'end_char': 2803}, {'period': '2016', 'start_char': 3615, 'end_char': 3619}, {'period': '2015', 'start_char': 3802, 'end_char': 3806}, {'period': 'July 2013', 'start_char': 4046, 'end_char': 4055}]
---------------------------------------------- 
FileName :  Usha Prabhakar- CV (ISHWA).pdf
Time Period:  [{'period': '2006 2011', 'start_char': 821, 'end_char': 830}, {'period': '2012', 'start_char': 1302, 'end_char': 1306}, {'period': 'Jan 2006', 'start_char': 2605, 'end_char': 2613}, {'period': 'July-Dec 2012', 'start_char': 2764, 'end_char': 2777}, {'period': 'June 1995', 'start_char': 4614, 'end_char': 4623}, {'period': '2002', 'start_char': 4670, 'end_char': 4674}, {'period': '2002', 'start_char': 4745, 'end_char': 4749}, {'period': '2006', 'start_char': 4789, 'end_char': 4793}, {'period': '2013 M. Phil', 'start_char': 4840, 'end_char': 4852}, {'period': '12th Apr

Time Period:  [{'period': 'October 2007 - December 2009', 'start_char': 597, 'end_char': 625}, {'period': 'March 2007 - October 2007', 'start_char': 706, 'end_char': 731}, {'period': 'June 2005 - March 2007', 'start_char': 789, 'end_char': 811}, {'period': '2000 - 2005', 'start_char': 867, 'end_char': 878}, {'period': '1999 - 2000', 'start_char': 939, 'end_char': 950}, {'period': '1996 - 1999', 'start_char': 1010, 'end_char': 1021}, {'period': '1994 - 1997', 'start_char': 1062, 'end_char': 1073}]
---------------------------------------------- 
FileName :  Vikas Mittal- CV.pdf
Time Period:  [{'period': '2012', 'start_char': 955, 'end_char': 959}, {'period': '2017', 'start_char': 986, 'end_char': 990}, {'period': '2001', 'start_char': 1421, 'end_char': 1425}, {'period': 'the year 2011', 'start_char': 1531, 'end_char': 1544}, {'period': 'February 1977', 'start_char': 1927, 'end_char': 1940}, {'period': '2007', 'start_char': 2254, 'end_char': 2258}, {'period': 'July 2012', 'start_char': 28

Time Period:  [{'period': 'August 2015', 'start_char': 340, 'end_char': 351}, {'period': '2014', 'start_char': 483, 'end_char': 487}, {'period': 'August 2015', 'start_char': 533, 'end_char': 544}, {'period': 'August 2015', 'start_char': 617, 'end_char': 628}, {'period': 'July 2015', 'start_char': 965, 'end_char': 974}, {'period': 'August 2014 to July 2015', 'start_char': 1047, 'end_char': 1071}, {'period': '2011', 'start_char': 1257, 'end_char': 1261}, {'period': 'July 2014', 'start_char': 1267, 'end_char': 1276}, {'period': 'November 2008-Apr 2011', 'start_char': 1998, 'end_char': 2020}, {'period': 'the financial year 2008-09', 'start_char': 2452, 'end_char': 2478}, {'period': 'June 2007', 'start_char': 2664, 'end_char': 2673}, {'period': 'October 2008', 'start_char': 2679, 'end_char': 2691}, {'period': 'May 2005', 'start_char': 3083, 'end_char': 3091}, {'period': 'May 2005', 'start_char': 3203, 'end_char': 3211}, {'period': 'Financial Year 2005-06', 'start_char': 3494, 'end_char': 35

#### Linking Company - Designation - Period

In [210]:
def get_full_experience(text, company_output, title_output, date_output):
    """
    Function to Link Company, Title & Period based on distances
    
    a) Input: Clean Text, Extracted Companies, Title & Dates
    
    b) Output: Mapped Company, Title & Dates
    """
    clean_txt = clean_special_text(text)
    
    out_ls = []
    if (len(company_output) > 0) and (len(title_output) > 0):
        for ls_1 in company_output:
            try:
                ls = []
                for ls_2 in title_output:
                    _dict = {'company_name':ls_1['keyword'], 'title':ls_2['keyword'], 'title_distance':'', 
                             'company_start_idx':ls_1['start_idx'], 'company_end_idx':ls_1['end_idx'],
                             'title_start_idx':ls_2['start_idx'], 'title_end_idx':ls_2['end_idx']}
                    company_end_idx = ls_1['end_idx']
                    title_start_idx = ls_2['start_idx']
                    _dict['title_distance'] = np.abs(company_end_idx-title_start_idx)
                    if (company_end_idx > title_start_idx):
                        _dict['title_words'] = len(clean_txt[title_start_idx:company_end_idx].strip().split())
                    else:
                        _dict['title_words'] = len(clean_txt[company_end_idx:title_start_idx].strip().split())

                    if _dict['title_words'] > 50: ## If # of words between company & title exceed 50 then don't assign [Assumption]
                        continue
                    ls.append(_dict)
                out_ls.append(min(ls, key=lambda x:x['title_distance']))
            except Exception as e:
                continue
        out_ls = sorted(out_ls, key = lambda i: (i['company_name'], i['title_words']), reverse=False)
        out_ls = list({i['company_name']:i for i in reversed(out_ls)}.values()) ## Remove dups
    else:
        return []
    
    full_ls = []

    if (len(out_ls) > 0) and (len(date_output) > 0):
        for ls_1 in out_ls:
            try:
                ls = []
                for ls_2 in date_output:
                    _dict = {'company_name':ls_1['company_name'], 'title':ls_1['title'], 'period':ls_2['period'], 'title_distance':ls_1['title_distance'], 
                             'date_distance':'', 'title_words':ls_1['title_words']}
                    company_end_idx = ls_1['company_end_idx']
                    date_start_idx = ls_2['start_idx']
                    _dict['date_distance'] = np.abs(company_end_idx-date_start_idx)

                    if (company_end_idx > date_start_idx):
                        _dict['date_words'] = len(clean_txt[date_start_idx:company_end_idx].strip().split())
                    else:
                        _dict['date_words'] = len(clean_txt[company_end_idx:date_start_idx].strip().split())

                    if _dict['date_words'] > 50: ## If # of words between company & date exceed 50 then don't assign [Assumption]
                        _dict['period'] = ''

                    ls.append(_dict)
                full_ls.append(min(ls, key=lambda x:x['date_distance']))
            except Exception as e:
                continue
    
        other_companies = list(set([i['keyword'] for i in company_output]) - set([i['company_name'] for i in full_ls]))

        other_ls = []
        for comp in other_companies:
            _dict = {'company_name': '', 
                     'title': '',
                     'period': '',
                     'title_distance': '',
                     'date_distance': '',
                     'title_words': '',
                     'date_words': ''}
            _dict['company_name'] = comp
            other_ls.append(_dict)

        final_ls = full_ls + other_ls
        
        return final_ls
    else:
        return out_ls

In [211]:
# file=os.listdir(RAW_DATA_PATH)[0]

# text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))

# company_output = get_company_names(text)
# title_output = get_job_titles(text)
# date_output = get_period(text)

In [212]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print(">> FileName : ",file)
    print("\n")
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    company_output = get_company_names(text)
    title_output = get_job_titles(text)
    date_output = get_period(text)
    experience = get_full_experience(text, company_output, title_output, date_output)
    print(">> Companies: ", company_output)
    print("\n")
    print(">> Designation: ", title_output)
    print("\n")
    print(">> Experience: ", experience, end="\n---------------------------------------------- \n")

>> FileName :  Abha Doshi- CV.pdf


>> Companies:  [{'keyword': 'ambit pragma advisors llp', 'start_idx': 1002, 'end_idx': 1027}, {'keyword': 'kpmg', 'start_idx': 3839, 'end_idx': 3843}, {'keyword': 'kpmg', 'start_idx': 3873, 'end_idx': 3877}, {'keyword': 'godrej hershey', 'start_idx': 5363, 'end_idx': 5377}, {'keyword': 'godrej group', 'start_idx': 5486, 'end_idx': 5498}, {'keyword': 'the hershey company', 'start_idx': 5525, 'end_idx': 5544}, {'keyword': 'the hershey company', 'start_idx': 6548, 'end_idx': 6567}, {'keyword': 'kpmg', 'start_idx': 7797, 'end_idx': 7801}, {'keyword': 'godrej group', 'start_idx': 8079, 'end_idx': 8091}, {'keyword': 'fine line circuits limited', 'start_idx': 8409, 'end_idx': 8435}, {'keyword': 'shah associates', 'start_idx': 8488, 'end_idx': 8503}]


>> Designation:  [{'keyword': 'finance professional', 'start_idx': 59, 'end_idx': 79}, {'keyword': 'senior consultant', 'start_idx': 423, 'end_idx': 440}, {'keyword': 'project manager', 'start_idx': 2711, 'end

>> Companies:  [{'keyword': 'o3 capital global advisory', 'start_idx': 342, 'end_idx': 368}, {'keyword': 'moody s analytics', 'start_idx': 786, 'end_idx': 803}]


>> Designation:  [{'keyword': 'analyst', 'start_idx': 315, 'end_idx': 322}, {'keyword': 'associate', 'start_idx': 757, 'end_idx': 766}, {'keyword': 'analyst', 'start_idx': 2835, 'end_idx': 2842}, {'keyword': 'chartered financial analyst', 'start_idx': 3309, 'end_idx': 3336}, {'keyword': 'event manager', 'start_idx': 4410, 'end_idx': 4423}]


>> Experience:  [{'company_name': 'o3 capital global advisory', 'title': 'analyst', 'period': 'analyst investment banking o3 capital global advisory may 2017 present', 'title_distance': 53, 'date_distance': 53, 'title_words': 7, 'date_words': 7}, {'company_name': 'moody s analytics', 'title': 'associate', 'period': 'associate investment banking moody s analytics october 2014 may 2017', 'title_distance': 46, 'date_distance': 46, 'title_words': 6, 'date_words': 6}]
-------------------------

>> Companies:  [{'keyword': 'reckitt benckiser', 'start_idx': 679, 'end_idx': 696}, {'keyword': 'paras pharmaceuticals', 'start_idx': 2715, 'end_idx': 2736}]


>> Designation:  [{'keyword': 'finance professional', 'start_idx': 95, 'end_idx': 115}, {'keyword': 'chartered accountant', 'start_idx': 488, 'end_idx': 508}, {'keyword': 'company secretary', 'start_idx': 594, 'end_idx': 611}, {'keyword': 'senior manager', 'start_idx': 996, 'end_idx': 1010}, {'keyword': 'senior manager', 'start_idx': 1202, 'end_idx': 1216}, {'keyword': 'manager', 'start_idx': 1376, 'end_idx': 1383}, {'keyword': 'management assistant', 'start_idx': 1532, 'end_idx': 1552}, {'keyword': 'manager', 'start_idx': 1553, 'end_idx': 1560}, {'keyword': 'commercial manager', 'start_idx': 3328, 'end_idx': 3346}, {'keyword': 'commercial manager', 'start_idx': 3559, 'end_idx': 3577}, {'keyword': 'factory manager', 'start_idx': 4385, 'end_idx': 4400}, {'keyword': 'secretary', 'start_idx': 5038, 'end_idx': 5047}, {'keyword': 'pr

>> Companies:  [{'keyword': 'unitary helion ventures', 'start_idx': 349, 'end_idx': 372}, {'keyword': 'global c', 'start_idx': 450, 'end_idx': 458}, {'keyword': 'oliver wyman', 'start_idx': 894, 'end_idx': 906}, {'keyword': 'ernst young', 'start_idx': 2391, 'end_idx': 2402}]


>> Designation:  [{'keyword': 'investment associate', 'start_idx': 107, 'end_idx': 127}, {'keyword': 'venture capital analyst', 'start_idx': 415, 'end_idx': 438}, {'keyword': 'summer associate', 'start_idx': 941, 'end_idx': 957}, {'keyword': 'analyst', 'start_idx': 1547, 'end_idx': 1554}, {'keyword': 'analyst', 'start_idx': 1561, 'end_idx': 1568}, {'keyword': 'banking analyst', 'start_idx': 2045, 'end_idx': 2060}, {'keyword': 'intern', 'start_idx': 2061, 'end_idx': 2067}, {'keyword': 'director', 'start_idx': 2276, 'end_idx': 2284}, {'keyword': 'summer associate', 'start_idx': 2403, 'end_idx': 2419}]


>> Experience:  [{'company_name': 'unitary helion ventures', 'title': 'venture capital analyst', 'period': 'augus

>> Companies:  [{'keyword': 'bharti airtel limited', 'start_idx': 2806, 'end_idx': 2827}, {'keyword': 'tulip it services', 'start_idx': 7189, 'end_idx': 7206}, {'keyword': 'accel icim systems and services limited', 'start_idx': 7287, 'end_idx': 7326}, {'keyword': 'enterprise and netwok solutions', 'start_idx': 7367, 'end_idx': 7398}, {'keyword': 'airtel', 'start_idx': 7664, 'end_idx': 7670}, {'keyword': 'airtel', 'start_idx': 7837, 'end_idx': 7843}, {'keyword': 'airtel', 'start_idx': 7899, 'end_idx': 7905}]


>> Designation:  [{'keyword': 'sales and marketing professional', 'start_idx': 96, 'end_idx': 128}, {'keyword': 'sales operations', 'start_idx': 2529, 'end_idx': 2545}, {'keyword': 'financial management', 'start_idx': 2677, 'end_idx': 2697}, {'keyword': 'manager enterprise', 'start_idx': 2869, 'end_idx': 2887}, {'keyword': 'account general manager', 'start_idx': 2950, 'end_idx': 2973}, {'keyword': 'team lead', 'start_idx': 3155, 'end_idx': 3164}, {'keyword': 'general manager', 'st

...

>> Companies:  [{'keyword': 'deloitte', 'start_idx': 1025, 'end_idx': 1033}, {'keyword': 'kpmg', 'start_idx': 1849, 'end_idx': 1853}, {'keyword': 'tata steel', 'start_idx': 2285, 'end_idx': 2295}, {'keyword': 'srei infrastructure finance limited', 'start_idx': 3545, 'end_idx': 3580}, {'keyword': 'bnp paribas', 'start_idx': 3581, 'end_idx': 3592}, {'keyword': 'computer point limited', 'start_idx': 3615, 'end_idx': 3637}, {'keyword': 'hdfc life', 'start_idx': 3934, 'end_idx': 3943}]


>> Designation:  [{'keyword': 'director', 'start_idx': 381, 'end_idx': 389}, {'keyword': 'senior analyst', 'start_idx': 1076, 'end_idx': 1090}, {'keyword': 'analyst', 'start_idx': 1888, 'end_idx': 1895}, {'keyword': 'acquisition analyst', 'start_idx': 2608, 'end_idx': 2627}, {'keyword': 'client representative', 'start_idx': 2979, 'end_idx': 3000}, {'keyword': 'project management intern', 'start_idx': 3094, 'end_idx': 3119}, {'keyword': 'director', 'start_idx': 3651, 'end_idx': 3659}]


>> Experience:  [{'co

>> Companies:  [{'keyword': 'delphi automotive systems', 'start_idx': 3225, 'end_idx': 3250}, {'keyword': 'optimum solutions', 'start_idx': 10223, 'end_idx': 10240}, {'keyword': 'optimal solutions', 'start_idx': 15991, 'end_idx': 16008}, {'keyword': 'delphi automotive systems', 'start_idx': 19988, 'end_idx': 20013}, {'keyword': 'delphi automotive systems', 'start_idx': 20170, 'end_idx': 20195}, {'keyword': 'parkash international', 'start_idx': 20964, 'end_idx': 20985}, {'keyword': 'reserve bank of india', 'start_idx': 22473, 'end_idx': 22494}]


>> Designation:  [{'keyword': 'finance professional', 'start_idx': 35, 'end_idx': 55}, {'keyword': 'director', 'start_idx': 539, 'end_idx': 547}, {'keyword': 'cfo', 'start_idx': 565, 'end_idx': 568}, {'keyword': 'cfo', 'start_idx': 815, 'end_idx': 818}, {'keyword': 'chartered accountant', 'start_idx': 1352, 'end_idx': 1372}, {'keyword': 'chartered accountant', 'start_idx': 2043, 'end_idx': 2063}, {'keyword': 'director', 'start_idx': 2775, 'end_

>> Companies:  [{'keyword': 'dupont pioneer', 'start_idx': 341, 'end_idx': 355}, {'keyword': 'phi seeds', 'start_idx': 8065, 'end_idx': 8074}, {'keyword': 'tata chemicals', 'start_idx': 9659, 'end_idx': 9673}]


>> Designation:  [{'keyword': 'general manager', 'start_idx': 3752, 'end_idx': 3767}, {'keyword': 'sales marketing', 'start_idx': 3871, 'end_idx': 3886}, {'keyword': 'business manager', 'start_idx': 5610, 'end_idx': 5626}, {'keyword': 'development lead', 'start_idx': 5695, 'end_idx': 5711}, {'keyword': 'business analyst', 'start_idx': 5718, 'end_idx': 5734}, {'keyword': 'sales development manager', 'start_idx': 8128, 'end_idx': 8153}, {'keyword': 'regional business manager', 'start_idx': 8193, 'end_idx': 8218}, {'keyword': 'business development manager', 'start_idx': 8280, 'end_idx': 8308}, {'keyword': 'sales development manager', 'start_idx': 8315, 'end_idx': 8340}, {'keyword': 'product manager', 'start_idx': 10394, 'end_idx': 10409}, {'keyword': 'area manager', 'start_idx': 1

>> Companies:  [{'keyword': 'capital ltd', 'start_idx': 400, 'end_idx': 411}, {'keyword': 'capital ltd', 'start_idx': 977, 'end_idx': 988}, {'keyword': 'capital ltd', 'start_idx': 2656, 'end_idx': 2667}, {'keyword': 'icici bank', 'start_idx': 3056, 'end_idx': 3066}, {'keyword': 'icici bank', 'start_idx': 3088, 'end_idx': 3098}, {'keyword': 'tata motors finance', 'start_idx': 3960, 'end_idx': 3979}, {'keyword': 'capital ltd', 'start_idx': 4007, 'end_idx': 4018}, {'keyword': 'tata motors', 'start_idx': 4526, 'end_idx': 4537}, {'keyword': 'boston consulting group', 'start_idx': 4599, 'end_idx': 4622}]


>> Designation:  [{'keyword': 'hr generalist', 'start_idx': 25, 'end_idx': 38}, {'keyword': 'hr generalist', 'start_idx': 999, 'end_idx': 1012}, {'keyword': 'organizational development specialist', 'start_idx': 1181, 'end_idx': 1218}, {'keyword': 'avp', 'start_idx': 1714, 'end_idx': 1717}, {'keyword': 'senior manager', 'start_idx': 1889, 'end_idx': 1903}, {'keyword': 'area sales manager', 

>> Companies:  [{'keyword': 'mape advisory group', 'start_idx': 106, 'end_idx': 125}, {'keyword': 'grant thornton', 'start_idx': 1709, 'end_idx': 1723}]


>> Designation:  [{'keyword': 'analyst', 'start_idx': 126, 'end_idx': 133}, {'keyword': 'internship', 'start_idx': 1324, 'end_idx': 1334}, {'keyword': 'tax consultant', 'start_idx': 1957, 'end_idx': 1971}]


>> Experience:  [{'company_name': 'mape advisory group', 'title': 'analyst', 'period': 'sept 16 present', 'title_distance': 1, 'date_distance': 19, 'title_words': 0, 'date_words': 3}, {'company_name': 'grant thornton', 'title': 'tax consultant', 'period': '', 'title_distance': 234, 'date_distance': 491, 'title_words': 35, 'date_words': 77}]
---------------------------------------------- 
>> FileName :  Yatin Joshi- CV.pdf


>> Companies:  [{'keyword': 'ambuja cements', 'start_idx': 948, 'end_idx': 962}, {'keyword': 'fosroc chemicals india', 'start_idx': 2922, 'end_idx': 2944}, {'keyword': 'rdc concrete india', 'start_idx': 4195, 

### Education

#### College/School Extraction

In [213]:
file = "../2. Data/indian_colleges_list_v7.txt"
file = open(file, "r", encoding='utf-8')

COLLEGE_LS = [line.strip().lower() for line in file]

# college = list(set([clean_special_text(i) for i in college]))
# college = [i for i in college if len(i.split()) > 1]

# COLLEGE_LS = list(set(COLLEGE_LS) - set(['institute of technology', 'management studies', 'and technology', 'navi mumbai', 'b sc nursing', 'and college', 
# 'management research', 'college of technology and engineering']))

# with open('../2. Data/indian_colleges_list_v7.txt', 'w', encoding='utf-8') as f:
#     for item in COLLEGE_LS:
#         f.write("%s\n" % item)

In [214]:
## Initialize Keyword Processor:
COLLEGE_KEYWORDS_SEARCH = basic_keyword_extract(COLLEGE_LS, case_sensitive=False)

In [215]:
def get_college_names(text):
    text = clean_special_text(text)
    college_ls = COLLEGE_KEYWORDS_SEARCH.extract_from_text(text)
    return college_ls

In [21]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    college_output = get_college_names(text)    
    print("College: ", college_output, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
College:  [{'keyword': 'university of mumbai', 'start_idx': 7274, 'end_idx': 7294}, {'keyword': 'indian institute of management calcutta', 'start_idx': 7867, 'end_idx': 7906}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
College:  [{'keyword': 'nagpur university', 'start_idx': 4896, 'end_idx': 4913}, {'keyword': 'icai', 'start_idx': 4891, 'end_idx': 4895}, {'keyword': 'iim indore', 'start_idx': 693, 'end_idx': 703}]
---------------------------------------------- 
FileName :  Abhilash Pillai- CV.pdf
College:  [{'keyword': 'indian institute of management lucknow', 'start_idx': 4559, 'end_idx': 4597}, {'keyword': 'iim lucknow', 'start_idx': 1097, 'end_idx': 1108}]
---------------------------------------------- 
FileName :  Abhinaw Kumar- CV.pdf
College:  [{'keyword': 'indian institute of information technology allahabad', 'start_idx': 5706, 'end_idx': 5758}, {'keyword': 'indian institute of management indore', 'start_idx

College:  [{'keyword': 'bit mesra', 'start_idx': 2062, 'end_idx': 2071}, {'keyword': 'xavier institute of management india', 'start_idx': 1972, 'end_idx': 2008}]
---------------------------------------------- 
FileName :  Anshuman Mishra- CV.pdf
College:  [{'keyword': 'iit kharagpur', 'start_idx': 449, 'end_idx': 462}, {'keyword': 'iim lucknow', 'start_idx': 430, 'end_idx': 441}]
---------------------------------------------- 
FileName :  Anu Mishra- CV.pdf
College:  [{'keyword': 'iit kharagpur', 'start_idx': 6870, 'end_idx': 6883}]
---------------------------------------------- 
FileName :  Anuj Jain- CV.pdf
College:  [{'keyword': 'bit mesra', 'start_idx': 7363, 'end_idx': 7372}, {'keyword': 'birla institute of technology', 'start_idx': 6901, 'end_idx': 6930}, {'keyword': 'indian institute of management lucknow', 'start_idx': 6862, 'end_idx': 6900}]
---------------------------------------------- 
FileName :  Anuj Varma- CV.pdf
College:  [{'keyword': 'symbiosis college', 'start_idx': 6

College:  [{'keyword': 'institute of chartered accountants of india', 'start_idx': 645, 'end_idx': 688}, {'keyword': 'monjee college', 'start_idx': 405, 'end_idx': 419}]
---------------------------------------------- 
FileName :  Bishnu Sahu- CV.pdf
College:  [{'keyword': 'nit jamshedpur', 'start_idx': 804, 'end_idx': 818}, {'keyword': 'goa institute of management', 'start_idx': 740, 'end_idx': 767}]
---------------------------------------------- 
FileName :  Bobby Ramesh Kamath- CV (ISHWA).pdf
College:  [{'keyword': 'sikkim manipal university', 'start_idx': 11005, 'end_idx': 11030}, {'keyword': 'st thomas college of nursing', 'start_idx': 9261, 'end_idx': 9289}]
---------------------------------------------- 
FileName :  Chaitanya Shravanth- CV.pdf
College:  [{'keyword': 'army institute of technology', 'start_idx': 5885, 'end_idx': 5913}, {'keyword': 'iim ahmedabad', 'start_idx': 5760, 'end_idx': 5773}]
---------------------------------------------- 
FileName :  Chandrashekara Bhat P-

College:  [{'keyword': 'bangalore university', 'start_idx': 5296, 'end_idx': 5316}, {'keyword': 'bangalore medical college', 'start_idx': 5270, 'end_idx': 5295}]
---------------------------------------------- 
FileName :  Ezra Greenberg- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Feroz Zainulabdeen- CV.pdf
College:  [{'keyword': 'national institute of technology tiruchirappalli', 'start_idx': 894, 'end_idx': 942}]
---------------------------------------------- 
FileName :  Gajanan Nabar- CV.pdf
College:  [{'keyword': 'narsee monjee institute of management studies', 'start_idx': 1106, 'end_idx': 1151}]
---------------------------------------------- 
FileName :  Gaurav Khurana- CV (ISHWA).pdf
College:  [{'keyword': 'panjab university', 'start_idx': 11749, 'end_idx': 11766}]
---------------------------------------------- 
FileName :  GAURAV KOTHARI- CV.pdf
College:  [{'keyword': 'birla institute of technology and science pilani', 'start_idx': 3267, 'en

College:  [{'keyword': 'cfa institute', 'start_idx': 1292, 'end_idx': 1305}]
---------------------------------------------- 
FileName :  Kamal Bajaj- CV.pdf
College:  [{'keyword': 'delhi public school', 'start_idx': 579, 'end_idx': 598}]
---------------------------------------------- 
FileName :  KANISHKA CHAUDHARY- CV (Vito Talent Syndication).pdf
College:  [{'keyword': 'kolkata university', 'start_idx': 1232, 'end_idx': 1250}]
---------------------------------------------- 
FileName :  KANISHKA CHAUDHARY- CV (Vito).pdf
College:  [{'keyword': 'kolkata university', 'start_idx': 1232, 'end_idx': 1250}]
---------------------------------------------- 
FileName :  Kapil Hathi- CV.pdf
College:  [{'keyword': 'university of mumbai', 'start_idx': 1752, 'end_idx': 1772}, {'keyword': 'monjee college', 'start_idx': 1737, 'end_idx': 1751}]
---------------------------------------------- 
FileName :  KAPIL MODI- CV.pdf
College:  [{'keyword': 'iit kharagpur', 'start_idx': 3502, 'end_idx': 3515}, {'ke

College:  [{'keyword': 'university of mumbai', 'start_idx': 10142, 'end_idx': 10162}]
---------------------------------------------- 
FileName :  MITALI CHITRE- CV.pdf
College:  [{'keyword': 'mumbai university', 'start_idx': 6950, 'end_idx': 6967}, {'keyword': 'vidyalankar institute of technology', 'start_idx': 6914, 'end_idx': 6949}]
---------------------------------------------- 
FileName :  Monu Ratra- CV.pdf
College:  [{'keyword': 'dav college', 'start_idx': 2367, 'end_idx': 2378}, {'keyword': 'guru nanak dev university', 'start_idx': 2287, 'end_idx': 2312}, {'keyword': 'lal bahadur shastri institute of management', 'start_idx': 2120, 'end_idx': 2163}]
---------------------------------------------- 
FileName :  MR. KARAN BANSAL- CV.pdf
College:  [{'keyword': 'symbiosis college', 'start_idx': 5805, 'end_idx': 5822}]
---------------------------------------------- 
FileName :  Mr. Maruthi B.M- CV (EMA Partners International).pdf
College:  [{'keyword': 'central food technological resea

College:  [{'keyword': 'iit kanpur', 'start_idx': 106, 'end_idx': 116}, {'keyword': 'indian institute of technology', 'start_idx': 396, 'end_idx': 426}]
---------------------------------------------- 
FileName :  Piyush Kabra- CV (ISHWA).pdf
College:  [{'keyword': 'mumbai university', 'start_idx': 1300, 'end_idx': 1317}, {'keyword': 'icai', 'start_idx': 1250, 'end_idx': 1254}]
---------------------------------------------- 
FileName :  Piyush Nagori- CV (Micheal Page).pdf
College:  []
---------------------------------------------- 
FileName :  Prabhakar Sunder- CV.pdf
College:  [{'keyword': 'madras university', 'start_idx': 3995, 'end_idx': 4012}, {'keyword': 'institute of chartered accountants of india', 'start_idx': 3916, 'end_idx': 3959}]
---------------------------------------------- 
FileName :  PRADEEP JANARDHANAN- CV.pdf
College:  [{'keyword': 'shree kerala verma college thrissur', 'start_idx': 5132, 'end_idx': 5167}, {'keyword': 'business school new delhi', 'start_idx': 5064, '

College:  [{'keyword': 'cfa institute', 'start_idx': 1747, 'end_idx': 1760}, {'keyword': 'shri ram college of commerce', 'start_idx': 1683, 'end_idx': 1711}]
---------------------------------------------- 
FileName :  Rohan Nagpal- CV.pdf
College:  [{'keyword': 'delhi public school', 'start_idx': 3289, 'end_idx': 3308}]
---------------------------------------------- 
FileName :  Roopam Upadhyay- CV.pdf
College:  [{'keyword': 'national institute of industrial engineering', 'start_idx': 1087, 'end_idx': 1131}, {'keyword': 'indian institute of technology bombay', 'start_idx': 4139, 'end_idx': 4176}, {'keyword': 'iim lucknow', 'start_idx': 962, 'end_idx': 973}, {'keyword': 'iit bombay', 'start_idx': 291, 'end_idx': 301}]
---------------------------------------------- 
FileName :  Roselind Mathews- CV (ISHWA).pdf
College:  [{'keyword': 'shreemati nathibai damodar thackersey women s university', 'start_idx': 6992, 'end_idx': 7048}]
---------------------------------------------- 
FileName :  

College:  [{'keyword': 'university of mumbai', 'start_idx': 5492, 'end_idx': 5512}, {'keyword': 'indian school of business hyderabad', 'start_idx': 4378, 'end_idx': 4413}, {'keyword': 'indian institute of management ahmedabad', 'start_idx': 4229, 'end_idx': 4269}]
---------------------------------------------- 
FileName :  Shefali Bairaria Suri- CV.pdf
College:  [{'keyword': 'dav college', 'start_idx': 811, 'end_idx': 822}, {'keyword': 'symbiosis institute of management studies', 'start_idx': 750, 'end_idx': 791}]
---------------------------------------------- 
FileName :  Shekhar Tewari- CV.pdf
College:  [{'keyword': 'st josephs college', 'start_idx': 2015, 'end_idx': 2033}, {'keyword': 'sj college', 'start_idx': 1575, 'end_idx': 1585}]
---------------------------------------------- 
FileName :  Sheshadri Savalgi- CV.pdf
College:  [{'keyword': 'ms university baroda', 'start_idx': 6572, 'end_idx': 6592}, {'keyword': 'icai', 'start_idx': 6375, 'end_idx': 6379}]
-------------------------

College:  [{'keyword': 'college of engineering', 'start_idx': 2982, 'end_idx': 3004}, {'keyword': 'jai hind college', 'start_idx': 3047, 'end_idx': 3063}, {'keyword': 'mumbai university', 'start_idx': 3012, 'end_idx': 3029}]
---------------------------------------------- 
FileName :  TARANPREET SINGH- CV.pdf
College:  [{'keyword': 'amritsar college of engineering', 'start_idx': 5184, 'end_idx': 5215}, {'keyword': 'khalsa college amritsar', 'start_idx': 5231, 'end_idx': 5254}]
---------------------------------------------- 
FileName :  TEJAS A. RAVAL- CV.pdf
College:  [{'keyword': 'ms university baroda', 'start_idx': 6723, 'end_idx': 6743}, {'keyword': 'icai', 'start_idx': 6386, 'end_idx': 6390}]
---------------------------------------------- 
FileName :  Tejash Mahendra Tanna- CV.pdf
College:  [{'keyword': 'icai', 'start_idx': 6073, 'end_idx': 6077}]
---------------------------------------------- 
FileName :  Uddeeptabh Mishra- CV (Kelly Services).pdf
College:  [{'keyword': 'xavier ins

#### Degree Extraction

In [216]:
file = "../2. Data/degree_list_v3.txt"
file = open(file, "r", encoding='utf-8')

DEGREE_LS = [line.strip().lower() for line in file]

In [217]:
# DEGREE_LS = list(set([clean_special_text(i) for i in DEGREE_LS]))
# DEGREE_LS = [i for i in DEGREE_LS if len(i.split()) > 1]

# DEGREE_LS = list(set(DEGREE_LS) - set(['be']))

# with open('../2. Data/degree_list_v3.txt', 'w', encoding='utf-8') as f:
#     for item in DEGREE_LS:
#         f.write("%s\n" % item)

In [218]:
## Initialize Keyword Processor:
DEGREE_KEYWORDS_SEARCH = basic_keyword_extract(DEGREE_LS, case_sensitive=False)

In [219]:
def get_degree_names(text):
    text = clean_special_text(text)
    degree_ls = DEGREE_KEYWORDS_SEARCH.extract_from_text(text)
    return degree_ls

In [23]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    degree_output = get_degree_names(text)  
    print("Degree: ", degree_output, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Degree:  [{'keyword': 'mba', 'start_idx': 833, 'end_idx': 836}, {'keyword': 'accounting and finance', 'start_idx': 7345, 'end_idx': 7367}, {'keyword': 'bachelor of commerce', 'start_idx': 7324, 'end_idx': 7344}, {'keyword': 'master of commerce', 'start_idx': 7243, 'end_idx': 7261}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 4853, 'end_idx': 4858}, {'keyword': 'mba', 'start_idx': 681, 'end_idx': 684}, {'keyword': 'chartered accountant', 'start_idx': 122, 'end_idx': 142}]
---------------------------------------------- 
FileName :  Abhilash Pillai- CV.pdf
Degree:  [{'keyword': 'pgdm', 'start_idx': 5716, 'end_idx': 5720}]
---------------------------------------------- 
FileName :  Abhinaw Kumar- CV.pdf
Degree:  [{'keyword': 'b tech', 'start_idx': 5699, 'end_idx': 5705}, {'keyword': 'mba', 'start_idx': 5599, 'end_idx': 5602}, {'keyword': 'master of business administration', 'st

Degree:  [{'keyword': 'pgdm', 'start_idx': 6697, 'end_idx': 6701}]
---------------------------------------------- 
FileName :  Anuj Varma- CV.pdf
Degree:  [{'keyword': 'post graduate', 'start_idx': 7571, 'end_idx': 7584}, {'keyword': 'b com', 'start_idx': 6784, 'end_idx': 6789}, {'keyword': 'diploma', 'start_idx': 6622, 'end_idx': 6629}, {'keyword': 'mba', 'start_idx': 3305, 'end_idx': 3308}]
---------------------------------------------- 
FileName :  Anupam Jain- CV.pdf
Degree:  [{'keyword': 'mba', 'start_idx': 3523, 'end_idx': 3526}, {'keyword': 'pg', 'start_idx': 2153, 'end_idx': 2155}]
---------------------------------------------- 
FileName :  Anvaya Pratyush- CV.pdf
Degree:  [{'keyword': 'm tech', 'start_idx': 120, 'end_idx': 126}, {'keyword': 'b tech', 'start_idx': 109, 'end_idx': 115}]
---------------------------------------------- 
FileName :  APURVA RAI- CV.pdf
Degree:  [{'keyword': 'llb', 'start_idx': 72, 'end_idx': 75}, {'keyword': 'ba', 'start_idx': 69, 'end_idx': 71}]
---

Degree:  [{'keyword': 'b sc', 'start_idx': 477, 'end_idx': 481}]
---------------------------------------------- 
FileName :  Deepesh Purwani- CV (Kelly).pdf
Degree:  [{'keyword': 'b sc', 'start_idx': 477, 'end_idx': 481}]
---------------------------------------------- 
FileName :  Dhananjay Chithathoor- CV (ISHWA).pdf
Degree:  [{'keyword': 'bachelors in science', 'start_idx': 10805, 'end_idx': 10825}, {'keyword': 'pgdm', 'start_idx': 10735, 'end_idx': 10739}, {'keyword': 'mba', 'start_idx': 182, 'end_idx': 185}, {'keyword': 'bsc', 'start_idx': 178, 'end_idx': 181}]
---------------------------------------------- 
FileName :  Dhiraj Mangaramani- CV.pdf
Degree:  [{'keyword': 'bachelor of commerce', 'start_idx': 580, 'end_idx': 600}, {'keyword': 'chartered accountancy', 'start_idx': 493, 'end_idx': 514}]
---------------------------------------------- 
FileName :  Dilip Menda- CV (Ishwa).pdf
Degree:  [{'keyword': 'bachelor of engineering', 'start_idx': 7476, 'end_idx': 7499}, {'keyword': 'p

Degree:  [{'keyword': 'b tech', 'start_idx': 9730, 'end_idx': 9736}, {'keyword': 'mba', 'start_idx': 9613, 'end_idx': 9616}, {'keyword': 'master of business administration', 'start_idx': 9579, 'end_idx': 9612}]
---------------------------------------------- 
FileName :  Haresh Balani- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 3894, 'end_idx': 3899}, {'keyword': 'bachelor of commerce', 'start_idx': 3873, 'end_idx': 3893}, {'keyword': 'mba', 'start_idx': 3507, 'end_idx': 3510}]
---------------------------------------------- 
FileName :  Harsh A. Desai- CV.pdf
Degree:  [{'keyword': 'diploma', 'start_idx': 3389, 'end_idx': 3396}, {'keyword': 'bachelor of science in mechanical engineering', 'start_idx': 2998, 'end_idx': 3043}]
---------------------------------------------- 
FileName :  HARSH BUCHASIA- CV.pdf
Degree:  [{'keyword': 'bachelors in commerce', 'start_idx': 3211, 'end_idx': 3232}, {'keyword': 'company secretary', 'start_idx': 3130, 'end_idx': 3147}]
----------------------

Degree:  [{'keyword': 'chartered accountancy', 'start_idx': 2716, 'end_idx': 2737}, {'keyword': 'bachelor of commerce', 'start_idx': 2648, 'end_idx': 2668}, {'keyword': 'mba', 'start_idx': 2636, 'end_idx': 2639}]
---------------------------------------------- 
FileName :  Lakshay Kataria- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 2918, 'end_idx': 2923}, {'keyword': 'chartered accountancy', 'start_idx': 2857, 'end_idx': 2878}]
---------------------------------------------- 
FileName :  Laveesh Bhandari- CV.pdf
Degree:  [{'keyword': 'ba', 'start_idx': 2203, 'end_idx': 2205}, {'keyword': 'master of arts', 'start_idx': 2148, 'end_idx': 2162}, {'keyword': 'ph d', 'start_idx': 2105, 'end_idx': 2109}]
---------------------------------------------- 
FileName :  Laxman B D- CV (EMA Partners International).pdf
Degree:  [{'keyword': 'm sc', 'start_idx': 1579, 'end_idx': 1583}]
---------------------------------------------- 
FileName :  Linda Haskins- CV (Promed Health Recruitment).pdf
De

Degree:  [{'keyword': 'ba', 'start_idx': 19641, 'end_idx': 19643}, {'keyword': 'bachelor of arts', 'start_idx': 19624, 'end_idx': 19640}, {'keyword': 'post graduation', 'start_idx': 19539, 'end_idx': 19554}]
---------------------------------------------- 
FileName :  NEETU MAHESHWARI- CV.pdf
Degree:  []
---------------------------------------------- 
FileName :  Nelson DSouza- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 701, 'end_idx': 706}, {'keyword': 'chartered accountancy', 'start_idx': 622, 'end_idx': 643}]
---------------------------------------------- 
FileName :  Nelson Prabhakaran- CV (ISHWA).pdf
Degree:  [{'keyword': 'bms', 'start_idx': 4051, 'end_idx': 4054}, {'keyword': 'diploma', 'start_idx': 7847, 'end_idx': 7854}, {'keyword': 'b com', 'start_idx': 157, 'end_idx': 162}, {'keyword': 'mba', 'start_idx': 163, 'end_idx': 166}, {'keyword': 'pgdm', 'start_idx': 7617, 'end_idx': 7621}]
---------------------------------------------- 
FileName :  NIDHI KHADRIA- CV.pdf
Degre

Degree:  [{'keyword': 'b com', 'start_idx': 2828, 'end_idx': 2833}]
---------------------------------------------- 
FileName :  R. RAJASEKARAN- CV (EMA Partners International).pdf
Degree:  [{'keyword': 'm sc', 'start_idx': 3784, 'end_idx': 3788}, {'keyword': 'ph d', 'start_idx': 108, 'end_idx': 112}, {'keyword': 'bachelor of science in agriculture', 'start_idx': 4002, 'end_idx': 4036}, {'keyword': 'master of science', 'start_idx': 3668, 'end_idx': 3685}, {'keyword': 'doctor of philosophy', 'start_idx': 3195, 'end_idx': 3215}]
---------------------------------------------- 
FileName :  Radhika Kedia- CV.pdf
Degree:  [{'keyword': 'diploma', 'start_idx': 2800, 'end_idx': 2807}, {'keyword': 'b com', 'start_idx': 263, 'end_idx': 268}, {'keyword': 'chartered accountant', 'start_idx': 171, 'end_idx': 191}]
---------------------------------------------- 
FileName :  RAGHAV MANDHANA- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 464, 'end_idx': 469}, {'keyword': 'chartered accountant', 'st

Degree:  [{'keyword': 'mba', 'start_idx': 5519, 'end_idx': 5522}]
---------------------------------------------- 
FileName :  SANDEEP KHANDELWAL- CV.pdf
Degree:  [{'keyword': 'mba', 'start_idx': 6801, 'end_idx': 6804}]
---------------------------------------------- 
FileName :  SANDEEP RANJAN- CV.pdf
Degree:  [{'keyword': 'mba', 'start_idx': 6858, 'end_idx': 6861}, {'keyword': 'pgdm', 'start_idx': 6853, 'end_idx': 6857}]
---------------------------------------------- 
FileName :  Sanjay Gupta- CV.pdf
Degree:  [{'keyword': 'accounting and finance', 'start_idx': 4955, 'end_idx': 4977}, {'keyword': 'bcom', 'start_idx': 4950, 'end_idx': 4954}, {'keyword': 'bachelor of commerce', 'start_idx': 4929, 'end_idx': 4949}, {'keyword': 'llb', 'start_idx': 4827, 'end_idx': 4830}, {'keyword': 'bachelor of law', 'start_idx': 4811, 'end_idx': 4826}, {'keyword': 'chartered accountant', 'start_idx': 4707, 'end_idx': 4727}, {'keyword': 'company secretary', 'start_idx': 4635, 'end_idx': 4652}]
------------

Degree:  [{'keyword': 'bachelor of arts', 'start_idx': 9819, 'end_idx': 9835}, {'keyword': 'diploma', 'start_idx': 9628, 'end_idx': 9635}, {'keyword': 'post graduate', 'start_idx': 9614, 'end_idx': 9627}, {'keyword': 'advanced program in strategy for leaders', 'start_idx': 9503, 'end_idx': 9543}]
---------------------------------------------- 
FileName :  Snehamoy Mukherjee- CV.pdf
Degree:  [{'keyword': 'integrated msc', 'start_idx': 2675, 'end_idx': 2689}]
---------------------------------------------- 
FileName :  Snehamoy Mukhurjee- CV.pdf
Degree:  [{'keyword': 'integrated msc', 'start_idx': 2675, 'end_idx': 2689}]
---------------------------------------------- 
FileName :  Somesh Lal- CV (ISHWA).pdf
Degree:  [{'keyword': 'diploma', 'start_idx': 6399, 'end_idx': 6406}, {'keyword': 'bachelor of engineering', 'start_idx': 1395, 'end_idx': 1418}, {'keyword': 'mba', 'start_idx': 186, 'end_idx': 189}]
---------------------------------------------- 
FileName :  Sonal Varma- CV.pdf
Degree:

Degree:  [{'keyword': 'bachelors in business administration', 'start_idx': 943, 'end_idx': 979}, {'keyword': 'msc', 'start_idx': 249, 'end_idx': 252}]
---------------------------------------------- 
FileName :  Vijay Dhanuka- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 166, 'end_idx': 171}, {'keyword': 'company secretary', 'start_idx': 145, 'end_idx': 162}, {'keyword': 'chartered accountant', 'start_idx': 121, 'end_idx': 141}]
---------------------------------------------- 
FileName :  Vikas Agrawal- CV.pdf
Degree:  [{'keyword': 'ph d', 'start_idx': 15555, 'end_idx': 15559}, {'keyword': 'doctor of philosophy', 'start_idx': 15534, 'end_idx': 15554}]
---------------------------------------------- 
FileName :  Vikas Lal- CV.pdf
Degree:  [{'keyword': 'b com', 'start_idx': 993, 'end_idx': 998}, {'keyword': 'chartered accountant', 'start_idx': 933, 'end_idx': 953}]
---------------------------------------------- 
FileName :  Vikas Mittal- CV.pdf
Degree:  [{'keyword': 'chartered account

#### Linking College & Degree

In [221]:
def get_full_education(text, college_output, degree_output):
    """
    Function to Link College & Degree based on distances
    
    a) Input: Text, Extracted Colleges & Degree
    
    b) Output: Mapped Colleges, Degree
    """
    clean_txt = clean_special_text(text)
    
    out_ls = []
    if (len(college_output) > 0) and (len(degree_output) > 0):
        for ls_1 in college_output:
            try:
                ls = []
                for ls_2 in degree_output:
                    _dict = {'college':ls_1['keyword'], 'degree':ls_2['keyword'], 'degree_distance':'', 
                             'college_start_idx':ls_1['start_idx'], 'college_end_idx':ls_1['end_idx'],
                             'degree_start_idx':ls_2['start_idx'], 'degree_end_idx':ls_2['end_idx']}
                    college_end_idx = ls_1['end_idx']
                    degree_start_idx = ls_2['start_idx']
                    _dict['degree_distance'] = np.abs(college_end_idx-degree_start_idx)
                    if (college_end_idx > degree_start_idx):
                        _dict['degree_words'] = len(clean_txt[degree_start_idx:college_end_idx].strip().split())
                    else:
                        _dict['degree_words'] = len(clean_txt[college_end_idx:degree_start_idx].strip().split())

                    if _dict['degree_words'] > 50: ## If # of words between college & degree exceed 15 then don't assign [Assumption]
                        continue
                    ls.append(_dict)
                out_ls.append(min(ls, key=lambda x:x['degree_distance']))
            except Exception as e:
                continue
        out_ls = sorted(out_ls, key = lambda i: (i['college'], i['degree_words']), reverse=False)
        out_ls = list({i['college']:i for i in reversed(out_ls)}.values()) ## Remove dups
        
        other_colleges = list(set([i['keyword'] for i in college_output]) - set([i['college'] for i in out_ls]))

        other_ls = []
        for college in other_colleges:
            _dict = {'college': '', 
                     'degree': '',
                     'degree_distance': '',
                     'degree_words': ''}
            _dict['college'] = college
            other_ls.append(_dict)

        full_ls = out_ls + other_ls
        
        return full_ls
    else:
        return out_ls

In [222]:
file=os.listdir(RAW_DATA_PATH)[0]

text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
college_output = get_college_names(text)    
degree_output = get_degree_names(text)  
education = get_full_education(text, college_output, degree_output)

In [224]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print(">> FileName : ",file)
    print("\n")
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    college_output = get_college_names(text)    
    degree_output = get_degree_names(text)  
    education = get_full_education(text, college_output, degree_output)
    print(">> College: ", college_output)
    print("\n")
    print(">> Degree: ", degree_output)
    print("\n")
    print(">> Education: ", education, end="\n---------------------------------------------- \n")

>> FileName :  Abha Doshi- CV.pdf


>> College:  [{'keyword': 'university of mumbai', 'start_idx': 7274, 'end_idx': 7294}, {'keyword': 'university of mumbai', 'start_idx': 7368, 'end_idx': 7388}, {'keyword': 'indian institute of management calcutta', 'start_idx': 7867, 'end_idx': 7906}, {'keyword': 'university of mumbai', 'start_idx': 8169, 'end_idx': 8189}]


>> Degree:  [{'keyword': 'mba', 'start_idx': 833, 'end_idx': 836}, {'keyword': 'mba', 'start_idx': 6925, 'end_idx': 6928}, {'keyword': 'master of commerce', 'start_idx': 7243, 'end_idx': 7261}, {'keyword': 'bachelor of commerce', 'start_idx': 7324, 'end_idx': 7344}, {'keyword': 'accounting and finance', 'start_idx': 7345, 'end_idx': 7367}, {'keyword': 'mba', 'start_idx': 7962, 'end_idx': 7965}]


>> Education:  [{'college': 'university of mumbai', 'degree': 'bachelor of commerce', 'degree_distance': 30, 'college_start_idx': 7274, 'college_end_idx': 7294, 'degree_start_idx': 7324, 'degree_end_idx': 7344, 'degree_words': 4}, {'coll

>> College:  [{'keyword': 'icai', 'start_idx': 352, 'end_idx': 356}, {'keyword': 'icai', 'start_idx': 357, 'end_idx': 361}, {'keyword': 'icai', 'start_idx': 362, 'end_idx': 366}, {'keyword': 'icai', 'start_idx': 367, 'end_idx': 371}, {'keyword': 'mumbai university', 'start_idx': 372, 'end_idx': 389}]


>> Degree:  []


>> Education:  []
---------------------------------------------- 
>> FileName :  Aditya Bhatt- CV.pdf


>> College:  [{'keyword': 'indian institute of management lucknow', 'start_idx': 1951, 'end_idx': 1989}, {'keyword': 'cfa institute', 'start_idx': 13303, 'end_idx': 13316}, {'keyword': 'iim lucknow', 'start_idx': 13317, 'end_idx': 13328}, {'keyword': 'monjee college', 'start_idx': 13362, 'end_idx': 13376}]


>> Degree:  [{'keyword': 'pg', 'start_idx': 0, 'end_idx': 2}, {'keyword': 'pg', 'start_idx': 150, 'end_idx': 152}, {'keyword': 'pg', 'start_idx': 1248, 'end_idx': 1250}, {'keyword': 'pg', 'start_idx': 1808, 'end_idx': 1810}, {'keyword': 'mba', 'start_idx': 1938, 'e

>> College:  [{'keyword': 'iit kharagpur', 'start_idx': 117, 'end_idx': 130}, {'keyword': 'shiv nadar university', 'start_idx': 2169, 'end_idx': 2190}, {'keyword': 'indian institute of technology kharagpur', 'start_idx': 2618, 'end_idx': 2658}, {'keyword': 'cfa institute', 'start_idx': 2817, 'end_idx': 2830}]


>> Degree:  [{'keyword': 'mba', 'start_idx': 148, 'end_idx': 151}, {'keyword': 'pgdm', 'start_idx': 2507, 'end_idx': 2511}, {'keyword': 'b tech', 'start_idx': 2588, 'end_idx': 2594}, {'keyword': 'm tech', 'start_idx': 2606, 'end_idx': 2612}]


>> Education:  [{'college': 'shiv nadar university', 'degree': 'pgdm', 'degree_distance': 317, 'college_start_idx': 2169, 'college_end_idx': 2190, 'degree_start_idx': 2507, 'degree_end_idx': 2511, 'degree_words': 48}, {'college': 'indian institute of technology kharagpur', 'degree': 'm tech', 'degree_distance': 52, 'college_start_idx': 2618, 'college_end_idx': 2658, 'degree_start_idx': 2606, 'degree_end_idx': 2612, 'degree_words': 8}, {'co

>> College:  [{'keyword': 'royal college', 'start_idx': 1960, 'end_idx': 1973}]


>> Degree:  [{'keyword': 'diploma', 'start_idx': 1346, 'end_idx': 1353}, {'keyword': 'diploma', 'start_idx': 1605, 'end_idx': 1612}]


>> Education:  [{'college': 'royal college', 'degree': '', 'degree_distance': '', 'degree_words': ''}]
---------------------------------------------- 
>> FileName :  Anja Stodtmeister- CV (Promed Heath Recruitment).pdf


>> College:  [{'keyword': 'royal college', 'start_idx': 1960, 'end_idx': 1973}]


>> Degree:  [{'keyword': 'diploma', 'start_idx': 1346, 'end_idx': 1353}, {'keyword': 'diploma', 'start_idx': 1605, 'end_idx': 1612}]


>> Education:  [{'college': 'royal college', 'degree': '', 'degree_distance': '', 'degree_words': ''}]
---------------------------------------------- 
>> FileName :  Ankit Singla- CV.pdf


>> College:  [{'keyword': 'iit kharagpur', 'start_idx': 296, 'end_idx': 309}, {'keyword': 'iit kharagpur', 'start_idx': 5563, 'end_idx': 5576}]


>> Degree:

>> College:  [{'keyword': 'indian institute of technology', 'start_idx': 154, 'end_idx': 184}, {'keyword': 'iit delhi', 'start_idx': 212, 'end_idx': 221}, {'keyword': 'iit delhi', 'start_idx': 585, 'end_idx': 594}, {'keyword': 'iit delhi', 'start_idx': 5002, 'end_idx': 5011}, {'keyword': 'iit delhi', 'start_idx': 5298, 'end_idx': 5307}, {'keyword': 'iit delhi', 'start_idx': 5497, 'end_idx': 5506}]


>> Degree:  [{'keyword': 'b tech', 'start_idx': 109, 'end_idx': 115}, {'keyword': 'm tech', 'start_idx': 120, 'end_idx': 126}]


>> Education:  [{'college': 'indian institute of technology', 'degree': 'm tech', 'degree_distance': 64, 'college_start_idx': 154, 'college_end_idx': 184, 'degree_start_idx': 120, 'degree_end_idx': 126, 'degree_words': 9}, {'college': 'iit delhi', 'degree': 'm tech', 'degree_distance': 101, 'college_start_idx': 212, 'college_end_idx': 221, 'degree_start_idx': 120, 'degree_end_idx': 126, 'degree_words': 17}]
---------------------------------------------- 
>> FileNa

...

>> College:  [{'keyword': 'university of mysore', 'start_idx': 2430, 'end_idx': 2450}]


>> Degree:  []


>> Education:  []
---------------------------------------------- 
>> FileName :  Srinath M- CV (ISHWA).pdf


>> College:  [{'keyword': 'andhra university', 'start_idx': 1507, 'end_idx': 1524}]


>> Degree:  [{'keyword': 'mba', 'start_idx': 187, 'end_idx': 190}, {'keyword': 'mba', 'start_idx': 1419, 'end_idx': 1422}]


>> Education:  [{'college': 'andhra university', 'degree': 'mba', 'degree_distance': 105, 'college_start_idx': 1507, 'college_end_idx': 1524, 'degree_start_idx': 1419, 'degree_end_idx': 1422, 'degree_words': 16}]
---------------------------------------------- 
>> FileName :  Srinivasan Seshadri- CV (IKYA).pdf


>> College:  [{'keyword': 'iim bangalore', 'start_idx': 1153, 'end_idx': 1166}, {'keyword': 'anna university', 'start_idx': 1206, 'end_idx': 1221}, {'keyword': 'indian institute of management bangalore', 'start_idx': 6018, 'end_idx': 6058}, {'keyword': 'anna un

>> College:  [{'keyword': 'monjee college', 'start_idx': 8862, 'end_idx': 8876}]


>> Degree:  [{'keyword': 'mba', 'start_idx': 8703, 'end_idx': 8706}, {'keyword': 'post graduation', 'start_idx': 8762, 'end_idx': 8777}, {'keyword': 'b com', 'start_idx': 8849, 'end_idx': 8854}]


>> Education:  [{'college': 'monjee college', 'degree': 'b com', 'degree_distance': 27, 'college_start_idx': 8862, 'college_end_idx': 8876, 'degree_start_idx': 8849, 'degree_end_idx': 8854, 'degree_words': 5}]
---------------------------------------------- 
>> FileName :  Suraj Yadav- CV.pdf


>> College:  [{'keyword': 'university of mumbai', 'start_idx': 172, 'end_idx': 192}, {'keyword': 'university of mumbai', 'start_idx': 357, 'end_idx': 377}]


>> Degree:  [{'keyword': 'm com', 'start_idx': 213, 'end_idx': 218}, {'keyword': 'bachelor of commerce', 'start_idx': 411, 'end_idx': 431}, {'keyword': 'b com', 'start_idx': 456, 'end_idx': 461}]


>> Education:  [{'college': 'university of mumbai', 'degree': 'm com'

>> College:  []


>> Degree:  [{'keyword': 'mba', 'start_idx': 5081, 'end_idx': 5084}]


>> Education:  []
---------------------------------------------- 
>> FileName :  URVASHI MAHENDRU- CV.pdf


>> College:  [{'keyword': 'university of delhi', 'start_idx': 3055, 'end_idx': 3074}, {'keyword': 'university of delhi', 'start_idx': 3262, 'end_idx': 3281}]


>> Degree:  [{'keyword': 'b com', 'start_idx': 3075, 'end_idx': 3080}]


>> Education:  [{'college': 'university of delhi', 'degree': 'b com', 'degree_distance': 1, 'college_start_idx': 3055, 'college_end_idx': 3074, 'degree_start_idx': 3075, 'degree_end_idx': 3080, 'degree_words': 0}]
---------------------------------------------- 
>> FileName :  Usha Prabhakar- CV (ISHWA).pdf


>> College:  [{'keyword': 'delhi university', 'start_idx': 2061, 'end_idx': 2077}, {'keyword': 'all india institute of medical sciences', 'start_idx': 2256, 'end_idx': 2295}, {'keyword': 'all india institute of medical sciences', 'start_idx': 2751, 'end_idx': 

>> College:  [{'keyword': 'icai', 'start_idx': 928, 'end_idx': 932}, {'keyword': 'delhi university', 'start_idx': 976, 'end_idx': 992}]


>> Degree:  [{'keyword': 'chartered accountant', 'start_idx': 933, 'end_idx': 953}, {'keyword': 'b com', 'start_idx': 993, 'end_idx': 998}]


>> Education:  [{'college': 'icai', 'degree': 'chartered accountant', 'degree_distance': 1, 'college_start_idx': 928, 'college_end_idx': 932, 'degree_start_idx': 933, 'degree_end_idx': 953, 'degree_words': 0}, {'college': 'delhi university', 'degree': 'b com', 'degree_distance': 1, 'college_start_idx': 976, 'college_end_idx': 992, 'degree_start_idx': 993, 'degree_end_idx': 998, 'degree_words': 0}]
---------------------------------------------- 
>> FileName :  Vikas Mittal- CV.pdf


>> College:  [{'keyword': 'institute of chartered accountants of india', 'start_idx': 2073, 'end_idx': 2116}, {'keyword': 'kurukshetra university', 'start_idx': 2144, 'end_idx': 2166}]


>> Degree:  [{'keyword': 'chartered accountant

>> College:  [{'keyword': 'university of pune', 'start_idx': 3123, 'end_idx': 3141}, {'keyword': 'university of pune', 'start_idx': 3166, 'end_idx': 3184}]


>> Degree:  [{'keyword': 'mba', 'start_idx': 3142, 'end_idx': 3145}, {'keyword': 'b e', 'start_idx': 3185, 'end_idx': 3188}]


>> Education:  [{'college': 'university of pune', 'degree': 'mba', 'degree_distance': 1, 'college_start_idx': 3123, 'college_end_idx': 3141, 'degree_start_idx': 3142, 'degree_end_idx': 3145, 'degree_words': 0}]
---------------------------------------------- 


### Skills

In [42]:
file = "../2. Data/skills_clean_list_v2.txt"
file = open(file, "r", encoding='utf-8')

SKILL_LS = [line.strip().lower() for line in file]

In [43]:
## Initialize Keyword Processor:
SKILLS_KEYWORDS_SEARCH = basic_keyword_extract(SKILL_LS, case_sensitive=False)

In [44]:
def extract_skills(text):
    text = clean_special_text(text)
    skill_ls = SKILLS_KEYWORDS_SEARCH.extract_from_text(text)
    return skill_ls

In [46]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    skills_output = extract_skills(text)    
    print("Skills: ", list(set([i['keyword'].upper() for i in skills_output])), end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Skills:  ['INTERNATIONAL MANAGEMENT', 'HINDI', 'ONLINE PRESENCE', 'DUTCH', 'BUSINESS OPERATIONS', 'REPORTING', 'BUILDING', 'FINANCE', 'INVESTMENT', 'PAY', 'PROJECT', 'BUDGETING FORECASTING', 'MID MARKET', 'OPERATIONS', 'LIABILITY', 'COLD CHAIN', 'FINANCIAL ADVISORY', 'ACCOUNTANTS', 'VOLUNTARY', 'LED', 'FINANCIAL STATEMENTS', 'DUE DILIGENCE', 'TEASERS', 'EAGLE', 'SYNERGIES', 'GOLD', 'FRANCHISE', 'DEAL SOURCING', 'CONTROL ENVIRONMENT', 'TEAMS', 'ECONOMICS', 'INVESTMENTS', 'GENERAL', 'CIRCUITS', 'ORDER TO CASH', 'DOCUMENTATION', 'PET FOOD', 'COST EFFECTIVE', 'IFRS', 'FINANCIAL REPORTING', 'ASSESSMENT', 'CXO', 'DIAMONDS', 'MEASURES', 'ROAMBI', 'PRODUCT MIX', 'BOOKKEEPING', 'GUJARATI', 'INTERNAL CONTROLS', 'CLOSURE', 'FLEET', 'COMPONENTS', 'OPERATIONAL DUE DILIGENCE', 'DECISION MAKING', 'CLOUD', 'COORDINATED', 'LANGUAGES', 'ACADEMIC BACKGROUND', 'FIXED ASSETS', 'GROWTH CAPITAL', 'STUDIO', 'AUTOMOTIVE', 'GROSS MARGIN', 'CORPORATE FINANCE', 'INTERNAL AUDIT', 'FI

Skills:  ['COMPENSATION AND BENEFITS', 'HINDI', 'EMPLOYEE RELATIONS', 'PROGRAMMES', 'POLICIES PROCEDURES', 'REPORTING', 'COMPENSATION', 'BUDGETING', 'COOKING', 'PIPELINE', 'PAY', 'PERFORMANCE MANAGEMENT', 'MAN POWER', 'RECORD', 'PROJECT', 'COMPENSATION BENEFITS', 'MUTUAL FUNDS', 'TECHNICAL ADVISORY', 'BANDING', 'DESIGNATION', 'TVC', 'MMS', 'COMPENSATION BENCHMARKING', 'SECURITIES', 'CAPITAL MARKETS', 'NEGOTIATION', 'MARATHON', 'PAYROLL', 'BENCHMARKING', 'FINANCIAL SERVICES', 'STAFF TRAINING', 'HR TRANSFORMATION', 'PERSONNEL FILES', 'BUNGALOWS', 'VIZ', 'LEARNING', 'SOURCING', 'REASON', 'INTERVIEWING', 'FRAUD', 'ESIC', 'BARRIERS', 'MAIL', 'BIRTH', 'ASSET MANAGEMENT', 'BOARDING', 'RESEARCH', 'LANGUAGES', 'FORMULATION', 'CAMPUS', 'CHEMISTRY', 'REGULATIONS', 'WEB TECHNOLOGIES', 'GOVERNMENT', 'INVESTORS', 'LEGAL REQUIREMENTS', 'GRIEVANCES', 'OPERATIONAL TRANSFORMATION', 'HOBBIES', 'REVISIONS', 'ROAD', 'SURVEY', 'INFORMATION SECURITY', 'SALARY', 'INTELLIGENCE', 'RISK', 'RISK CONSULTING', 'ONB

Skills:  ['MARKET RESEARCH', 'WWW', 'AGENDA', 'RETAIL AUDIT', 'CIRCULATION', 'BRAND EQUITY', 'STRATEGIC THINKING', 'PROMO', 'CONSUMER PRODUCTS', 'MULTI COUNTRY', 'CONSUMER FOCUS', 'MARKETING LEADERSHIP', 'BUILDING', 'RESEARCH', 'PRODUCT INNOVATION', 'GENERAL ELECTRIC', 'MEDIA RESEARCH', 'SUPPLY CHAIN', 'NIELSEN', 'CAMPAIGNS', 'INSECTICIDES', 'MONEY', 'BRAND STRATEGY', 'RANGE', 'EMAIL', 'FINANCE', 'LAUNCHES', 'MAR', 'INSURANCE', 'FINISHED GOODS', 'MARKETING MIX', 'MEDIA MARKETING', 'PRODUCT QUALITY', 'APAC', 'PRODUCTION PLANNING', 'NET', 'GLOBAL SALES', 'PERSONAL CARE', 'TEAMS', 'DETERGENTS', 'ELEMENTS', 'DIGITAL', 'PROJECT', 'GENERAL', 'RECORD', 'DEMAND', 'ANALYTICS', 'DIARY', 'NEWSPAPERS', 'DIGITAL MARKETING', 'LEADERSHIP', 'FMCG', 'SOLUTION SELLING', 'HOUSEHOLD', 'PROFITABLE GROWTH', 'LED', 'MARKET SHARE']
---------------------------------------------- 
FileName :  Ajit Jangle- CV (HEDRICKS & STRUGGLES).pdf
Skills:  ['SALES PERFORMANCE', 'REACH', 'STRATEGIC THINKING', 'REPORTING', 'D

Skills:  ['COMPLEXITY REDUCTION', 'DATABASES', 'REPORTING', 'RANGE', 'BUDGETING', 'EDUCATION SOFTWARE', 'PHYSICS', 'FINANCE', 'PROGRAM MANAGEMENT', 'PHARMACEUTICALS', 'INVESTMENT', 'RESPONSE MODELING', 'RAN', 'FRENCH', 'MULTIPLES', 'OPERATING BUDGETS', 'DIRECTORS', 'PROJECT', 'RESOURCE ALLOCATION', 'OPERATIONS', 'LIABILITY', 'PRICING STRATEGY', 'LEADERSHIP', 'FMCG', 'CAPITAL', 'BLOOMBERG', 'AUTOMATION', 'LED', 'COST EFFICIENCY', 'EBITDA', 'DIRECT MARKETING', 'COACHING', 'LAKE', 'PROCESS MANAGEMENT', 'BUSINESS ANALYSIS', 'ROIC', 'HAIR CARE', 'LANDSCAPING', 'CAPACITY UTILIZATION', 'GOLD', 'LAUNCHES', 'STRUCTURAL', 'TEXAS', 'INVESTMENT BANKING', 'MATLAB', 'PERSONAL CARE', 'TEAMS', 'PERFORMANCE ANALYSIS', 'TRIALS', 'INVESTMENTS', 'REMODELING', 'SKIN CARE', 'AVG', 'VALUE CHAIN ANALYSIS', 'SPSS', 'ASSESSMENT', 'SHARED SERVICES', 'ORBIT', 'AUDIT COMMITTEE', 'FRAMEWORK', 'INTERNAL CONTROLS', 'OWNERSHIP', 'PREDICTION', 'BUSINESS PLANNING', 'TAX', 'SAVINGS', 'SUPPLY CHAIN', 'CORPORATE BUDGETING'

Skills:  ['WATER', 'ACT', 'STRATEGIC PLANNING', 'NRP', 'TESTING', 'BOX', 'NURSERY', 'INTENSIVE CARE', 'VIP', 'PUBLIC HEALTH', 'POLICIES PROCEDURES', 'CONSCIOUS SEDATION', 'HISTORY', 'DELEGATION', 'ASSISTANTS', 'GENERATORS', 'SECURITY', 'UROLOGY', 'FERTILITY', 'STAFF DEVELOPMENT', 'FINALIZATION', 'INTERPRETING', 'PACKING', 'POSITIVE WORK ENVIRONMENT', 'DASHBOARDS', 'ADMINISTRATIVE', 'PREVENTION', 'POLLUTION', 'FIRE TRAINING', 'TEACHING', 'RENOVATION', 'NEPHROLOGY', 'PROFESSIONAL DEVELOPMENT', 'MAN', 'INJURY', 'CONSUMABLES', 'ORGANIZATIONAL ETHICS', 'PATIENT EDUCATION', 'THEATERS', 'MICROSOFT WORD', 'RADIATION', 'CONSTRUCTION', 'FACILITATION', 'BARIATRICS', 'PERFORMANCE MANAGEMENT', 'SKYPE', 'RECORD', 'PROCEDURAL', 'REGISTRATION', 'LIFE SUPPORT', 'LAUNDRY', 'PROCEDURE MANUALS', 'MEDICAL EQUIPMENT', 'PRESSURE', 'FLOOD', 'ENERGY', 'CARDIOLOGY', 'PREPARATION', 'PROFESSIONAL ASSOCIATIONS', 'ACTING', 'LEADERSHIP', 'CAPITAL', 'AVAILABILITY', 'SPECIFICATIONS', 'RECORDS', 'DIALYSIS', 'HEALTH SCI

...

Skills:  ['UNION', 'COACHING', 'HINDI', 'INCOME TAX', 'OFFICERS', 'SAP ERP', 'ANIMAL HEALTH', 'RETAIL', 'DOCUMENTARIES', 'ELECTRONICS', 'COMMERCE', 'PROVISION', 'TAX PLANNING', 'BUILDING', 'FINANCIAL SYSTEMS', 'REPORTING', 'CHESS', 'TAX', 'EBITDA', 'LIQUIDITY', 'HEALTH', 'BUSINESS PLANNING', 'GERMAN', 'FINANCIAL STRATEGY', 'SAVINGS', 'LANGUAGES', 'HEALTHCARE', 'FINANCIAL STATEMENTS', 'USER TRAINING', 'BUDGETING', 'FIXED ASSETS', 'TRANSFER PRICING', 'CISA', 'SEM', 'FINANCE', 'STRUCTURAL', 'DIRECT TAX', 'VENDOR MANAGEMENT', 'LEARNING', 'BRANCH BANKING', 'BUSINESS PROCESS IMPROVEMENT', 'GROSS MARGIN', 'INVENTORY CONTROL', 'MANUFACTURING', 'TAX AUDITS', 'PLANTS', 'SDL', 'ORION', 'TAX DEDUCTED AT SOURCE TDS', 'INTERNAL AUDIT', 'PAPER', 'ITALIAN', 'COST REDUCTION', 'INFORMATION TECHNOLOGY', 'ISACA', 'BUDGETING FORECASTING', 'INCENTIVES', 'OPERATIONS', 'STRATEGY', 'BUSINESS UNITS', 'ENGLISH', 'STATUTORY AUDIT', 'IFRS', 'PREPARATION', 'ACCOUNT CREATION', 'SAP IMPLEMENTATION', 'VENDORS', 'RBI',

Skills:  ['LEVEL III', 'SHAPE', 'PUBLICATIONS', 'INSTITUTIONAL', 'MATERIALS', 'SENSITIVITY', 'TIER 2', 'MEDICARE', 'BENCHMARKING', 'MANAGEMENT CONSULTING', 'COMPANY VALUATION', 'TELECOM', 'TELECOMMUNICATIONS', 'SERVICE PROVIDERS', 'RESEARCH', 'OEMS', 'SILVER', 'ASIAN MARKETS', 'LAL', 'PERSONAL BANKING', 'CABLE', 'SMALL CAP', 'DEVELOPERS', 'TRADING DESK', 'TECHNOLOGY', 'SPACE', 'HOCKEY', 'INVESTMENT', 'FRANCHISE', 'INVESTORS', 'BUSINESS CASE', 'PIPELINE', 'SCENARIO ANALYSIS', 'SEGMENTATION', 'TRADING', 'ACCESS', 'HOSPITALS', 'APOLLO', 'WRITING', 'TEAMS', 'EVENTS', 'MULTIPLES', 'AUGMENTED REALITY', 'MEDIA TECHNOLOGY', 'GENERAL', 'STRATEGY', 'MARKET ENTRY', 'MODELS', 'EQUITIES', 'CONFERENCES', 'ENERGY', 'INVESTMENTS', 'STOCKS', 'SPECTRUM', 'BRONZE', 'BOUTIQUE', 'UNIVERSITIES', 'LEADERSHIP', 'DSL', 'FINANCIAL RISK', 'REVISIONS', 'ROAD', 'DDM', 'ASSESSMENT', 'WEB', 'CONSUMER ELECTRONICS', 'DCF', 'RATINGS', 'CASE STUDIES', 'HEDGE FUNDS', 'ADDITION', 'LOCATION BASED SERVICES', 'FRAMEWORK', 'C

Skills:  ['ACT', 'STRATEGIC PLANNING', 'REACH', 'YAHOO', 'INCOME TAX', 'CHINESE', 'API', 'INDICATORS', 'REPORTING', 'APPEALS', 'BUILDING', 'CONTINUOUS IMPROVEMENT', 'FINALIZATION', 'BUDGETING', 'FINANCE', 'PHARMACEUTICALS', 'TEACHING', 'FINANCIAL PERFORMANCE', 'INVESTMENT', 'DIRECT TAX', 'QRM', 'CAREER DEVELOPMENT', 'EXIST', 'PROJECT', 'OPERATIONS', 'MECHANISMS', 'REAL ESTATE', 'ENERGY', 'PREPARATION', 'FRAMING', 'LEADERSHIP', 'CALCULATIONS', 'CASH FLOW', 'TRIBUNALS', 'CAPITAL', 'ACCOUNTANTS', 'EMBEDDED', 'SALES OPERATIONS', 'EBITDA', 'COACHING', 'PERFORMANCE REPORTING', 'ROLLING', 'FINANCIALS', 'COST MANAGEMENT', 'DUE DILIGENCE', 'BENCHMARKING', 'LEGAL ISSUES', 'FINANCIAL OPERATIONS', 'SALES GROWTH', 'MONEY', 'GARDEN', 'HUMILITY', 'MITIGATION', 'PURCHASING', 'GENERAL MOTORS', 'CONTROL ENVIRONMENT', 'CUSTOMER PROFITABILITY', 'APPROVALS', 'TEAMS', 'GENERAL', 'DOCUMENTATION', 'BUSINESS PROFITABILITY', 'IFRS', 'DIVIDENDS', 'ASSESSMENT', 'VALIDATION', 'CREDIT', 'BIRTH', 'POWERTRAIN', 'FRAM

Skills:  ['INCREASE PRODUCTIVITY', 'STRATEGIC PLANNING', 'BUSINESS PROCESS', 'FINALIZATION', 'INTERPRETING', 'BUDGETING', 'DASHBOARDS', 'INVOICING', 'PROBABILITY', 'CHART', 'FINANCE', 'FINANCIAL PERFORMANCE', 'FOREIGN CURRENCY', 'INVESTMENT', 'PIPELINE', 'DRAWING', 'DEVELOPMENT CENTERS', 'LOANS', 'RECORD', 'PROJECT', 'RESOURCE ALLOCATION', 'OPEX', 'OPERATIONS', 'PROCESS VALIDATION', 'UTILIZATION REVIEW', 'SG A', 'PRICING STRATEGY', 'PREPARATION', 'LEADERSHIP', 'IRR', 'CASH FLOW', 'CAPITAL', 'REVIEWS', 'ACCOUNTANTS', 'LONG RANGE PLANNING', 'MACRO', 'FINANCIAL STATEMENTS', 'KEY PERFORMANCE INDICATORS', 'FINANCIALS', 'LOAN', 'NUTS', 'BENCHMARKING', 'NEW STORE OPENINGS', 'LAUNCHES', 'STRUCTURAL', 'VIZ', 'FLASH', 'LEARNING', 'LIQUIDATION', 'FINANCIAL DATA', 'PROCESS EFFICIENCY', 'TEAM MANAGEMENT', 'TEAMS', 'GENERAL', 'FINANCIAL REPORTING', 'MEASURES', 'CREDIT', 'SUBSIDIARIES', 'BIRTH', 'MANAGEMENT PROFESSIONAL', 'FRAMEWORK', 'ADHERENCE', 'PROMPT', 'WORKING CAPITAL MANAGEMENT', 'STORE', 'PEO

Skills:  ['POLYMER COMPOUNDING', 'DUE DILIGENCE', 'ART', 'MANDATES', 'SELL SIDE', 'HEALTHCARE ANALYTICS', 'TRA', 'BUILDING', 'SME', 'RESEARCH', 'TAX', 'REIKI', 'COMPLETION', 'POLYPROPYLENE', 'CRYPTO', 'DCF VALUATION', 'INVESTORS', 'COMPOUNDING', 'SCENARIO ANALYSIS', 'PROCESS ENGINEERING', 'CURRENCY', 'MODELS', 'BASE', 'NBFC', 'LIABILITY', 'FACILITIES MANAGEMENT', 'VALUATION', 'CLASSIFICATION', 'PACKAGING', 'BIRTH', 'CREDIT RATING', 'FOUNDATION', 'RUBBER']
---------------------------------------------- 
FileName :  Yatin Joshi- CV.pdf
Skills:  ['WATER', 'WLL', 'CREDIT CONTROL', 'MATERIALS', 'SEALANTS', 'COMMUNICATIONS', 'NOTES', 'TIME MANAGEMENT', 'READY MIX CONCRETE', 'MOULD', 'APPLICATION', 'TUNNELING', 'PROTECTION', 'FLOORING', 'RESEARCH', 'BUILDING', 'MULTINATIONAL', 'TROUBLESHOOTING', 'RANGE', 'STRUCTURAL REPAIRS', 'AGGREGATES', 'PROTECTIVE COATINGS', 'RAPID GROWTH', 'CONSTRUCTION', 'RDC', 'PRINT MEDIA', 'PAPER', 'TEAM MANAGEMENT', 'PRECAST', 'PRODUCT LAUNCH', 'CONCRETE', 'SALES MA

### Table Extraction

In [275]:
file_path = r"C:\Users\admin\Downloads\nic_2008_17apr09.pdf"

In [276]:
import camelot

In [303]:
page_list = ",".join([str(i) for i in np.arange(36,138)])

In [305]:
table_list = camelot.read_pdf(file_path, 
                              pages=page_list, 
                              flavor="stream", 
                              flag_size=True,
                              line_size_scaling=80)

In [307]:
tab_df = pd.DataFrame()

In [308]:
tab_ls = []

In [309]:
for i in range(len(table_list)):
    tab_df = pd.concat([tab_df, table_list[i].df],
                      axis=0,
                      ignore_index=True).drop_duplicates()

In [355]:
tab_df.columns = ['col_1', 'col_2', 'col_3', 'col_4', 'col_5']

In [360]:
df = tab_df[["col_3", "col_4"]]

In [361]:
df = df.replace(r'^\s*$', np.nan, regex=True)

In [363]:
df.to_clipboard()

In [333]:
df["col_3"] = df["col_3"].ffill()

In [336]:
df = df.dropna()

In [340]:
df

,col_3,col_4
2,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",relateD ServiCe aCtivitieS
3,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",Growing of non-perennial crops
4,"DiviSion 01 : Crop anD animal proDuCtion, hunt...","Growing of cereals (except rice), leguminous c..."
5,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",This class includes all forms of growing of ce...
6,"DiviSion 01 : Crop anD animal proDuCtion, hunt...","oil seeds in open fields, including those cons..."
...,...,...
4294,98200,activities of extraterritorial organizations a...
4295,98200,This class includes the activities of internat...
4296,98200,"United Nations and its agencies, IMF, World Ba..."
4297,98200,OPEC etc.


In [347]:
df

,col_3,col_4
2,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",relateD ServiCe aCtivitieS
3,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",Growing of non-perennial crops
4,"DiviSion 01 : Crop anD animal proDuCtion, hunt...","Growing of cereals (except rice), leguminous c..."
5,"DiviSion 01 : Crop anD animal proDuCtion, hunt...",This class includes all forms of growing of ce...
6,"DiviSion 01 : Crop anD animal proDuCtion, hunt...","oil seeds in open fields, including those cons..."
...,...,...
4294,98200,activities of extraterritorial organizations a...
4295,98200,This class includes the activities of internat...
4296,98200,"United Nations and its agencies, IMF, World Ba..."
4297,98200,OPEC etc.


In [348]:
for i in df.index:
    df.loc[i, "flag"] = df.loc[i, "col_3"].isnumeric()

In [351]:
main_df = df[df['flag']==True]

In [354]:
main_df.groupby(['col_3'], as_index=False).agg(lambda x: " ".join(x)).to_clipboard()

In [335]:
df.to_clipboard()

In [323]:
df["col_3"].ffill()

0                                                        
1       DiviSion 01 : Crop anD animal proDuCtion, hunt...
2                                                        
3                                                        
4                                                        
                              ...                        
4294                                                     
4295                                                     
4296                                                     
4297                                                     
4298                                                99000
Name: col_3, Length: 4295, dtype: object

In [311]:
tab_df.to_clipboard()